# **Set Up Environmen**t

In [1]:
import csv
from google.colab import files
import numpy as np
import pandas as pd
import ast
import re

In [2]:
!pip install pyspark
!pip install -U -q PyDrive2
#the output 'xxx is not a symbolic link' will not affect your implementation or execution
#to fix 'xxx is not a symbolic link', you can comment out the lines starting from !mv xxxx
#you may need to replace xxx.11 with the correct version if other errors come up after colab update
#to get the correct version, use !ls /usr/local/lib to find out
!mv /usr/local/lib/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2.backup
!mv /usr/local/lib/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2.backup
!mv /usr/local/lib/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3.backup
!mv /usr/local/lib/libtbb.so.12 /usr/local/lib/libtbb.so.12.backup
!mv /usr/local/lib/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3.backup
!mv /usr/local/lib/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3.backup
!ln -s /usr/local/lib/libtbbmalloc_proxy.so.2.11 /usr/local/lib/libtbbmalloc_proxy.so.2
!ln -s /usr/local/lib/libtbbmalloc.so.2.11 /usr/local/lib/libtbbmalloc.so.2
!ln -s /usr/local/lib/libtbbbind_2_5.so.3.11 /usr/local/lib/libtbbbind_2_5.so.3
!ln -s /usr/local/lib/libtbb.so.12.11 /usr/local/lib/libtbb.so.12
!ln -s /usr/local/lib/libtbbbind_2_0.so.3.11 /usr/local/lib/libtbbbind_2_0.so.3
!ln -s /usr/local/lib/libtbbbind.so.3.11 /usr/local/lib/libtbbbind.so.3
# !sudo ldconfig
#If error related to the above execution occurs, you can try commenting out the above 12 lines under pip install PyDrive2 (not included)
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#the output 'xxx is not a symbolic link' will not affect your implementation or execution
#to fix 'xxx is not a symbolic link', you can comment out the lines starting from !mv xxxx
#you may need to replace xxx.11 with the correct version if other errors come up after colab update
#to get the correct version, use !ls /usr/local/lib to find out


The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 123634 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u432-ga~us1-0ubuntu2~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u432-ga~us1-0ub

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
from pyspark.sql.functions import col, udf
from pyspark.sql.types import BooleanType


# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

# **Clean Gene Expression Table**

In [ ]:
data = spark.read.csv('rna_single_cell_type_tissue.tsv', sep="\t", header=True, inferSchema=True)
# Only keep the surface genes
surface_gene = set(pd.read_csv('mass_spec_valid_surface_protein.csv')['ENTREZ gene symbol'])
gene_cell = data.filter(data['Gene name'].isin(surface_gene)).drop('Gene').drop('Read count')
# If same ('Tissue', 'Cell type', 'Gene name') has two clusters, pick the higher nTPM one
windowSpec = Window.partitionBy('Tissue', 'Cell type', 'Gene name').orderBy(desc('nTPM'))
gene_rank = gene_cell.withColumn('rank', row_number().over(windowSpec))
gene_ranked = gene_rank.filter(gene_rank['rank'] == 1).drop('rank')
tissue_cell_genes = gene_ranked.withColumn('gene_nTPM', struct('Gene name', 'nTPM')).groupBy('Tissue', 'Cell type').agg(collect_list("gene_nTPM"))
data_list = tissue_cell_genes.collect()
# Remove all the mixed cell types
mixed_cells_id = []
cleaned_data = []
for i in range(len(data_list)):
  if 'mixed' in data_list[i][1]:
    mixed_cells_id.append(i)
for i in range(len(data_list)):
  if i not in mixed_cells_id:
    cleaned_data.append(data_list[i])

Filtering for Genes Encoding Surface Proteins

In [ ]:
# Source 1: Public mass-spec data set derived surface proteins

membrane_protein_ccle = pd.read_csv("/content/drive/MyDrive/CCLE proteome/mass_spec_valid_surface_protein.csv", header=0, index_col=0)
memprot_ccle = membrane_protein_ccle["ENTREZ gene symbol"]

# Source 2: Uniprot derived membrane protein list

membrane_protein_uniprot = pd.read_csv("/content/drive/MyDrive/CCLE proteome/uniprot_subcellular_localization", sep="\t",header=0, index_col=0)

# remove the nan gene names
membrane_protein_uniprot_with_name = membrane_protein_uniprot.loc[membrane_protein_uniprot["name_NAN"]==True]

gene_name = []
for i, gene in enumerate(membrane_protein_uniprot_with_name["Gene Names"]):
  if gene==gene:
   gene_name.append(gene.split()[0])

membrane_protein_uniprot_with_name["Gene Name real"] = gene_name

# check if the processed gene_name is the same as in mass spec dataset
common_gene = set(list(memprot_ccle)).intersection(set(list(gene_name)))
common_gene = list(common_gene)
membrane_protein_uniprot_with_name.index = gene_name
# the majority of gene names in mass spec dataset is present in uniprot data set
# focus on those genes

# Some genes have more than one protein product, combining their annotation
# Only retaining genes with annotation of "Membrane" or "Cell Membrane":

non_membrane_gene=[]
for gene in common_gene:
  if type(membrane_protein_uniprot_with_name.loc[gene,"Subcellular location [CC]"]) != str:
    loca = ",".join(membrane_protein_uniprot_with_name.loc[gene,"Subcellular location [CC]"])

  else:
    loca = membrane_protein_uniprot_with_name.loc[gene,"Subcellular location [CC]"]

  if re.search("Cell membrane",loca) or re.search("Membrane",loca):
    continue
  else:
    #print(loca)
    non_membrane_gene.append([gene,loca])

pd.DataFrame(non_membrane_gene).to_csv("/content/drive/MyDrive/CCLE proteome/uniprot_non_membrane_gene")


Remove non-membrane genes

In [ ]:
string = ''',0,1
0,YBX1,"SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:12604611, ECO:0000269|PubMed:14718551, ECO:0000269|PubMed:15806160}. Nucleus {ECO:0000269|PubMed:12604611, ECO:0000269|PubMed:14718551, ECO:0000269|PubMed:15806160}. Cytoplasmic granule {ECO:0000269|PubMed:17289661, ECO:0000269|PubMed:18335541, ECO:0000269|PubMed:19029303, ECO:0000269|PubMed:25229427}. Secreted {ECO:0000269|PubMed:19483673}. Secreted, extracellular exosome {ECO:0000269|PubMed:27559612, ECO:0000269|PubMed:28341602}. Cytoplasm, P-body {ECO:0000250|UniProtKB:P62960}. Note=Predominantly cytoplasmic in proliferating cells (PubMed:12604611). Cytotoxic stress and DNA damage enhance translocation to the nucleus (PubMed:14718551). Localized in cytoplasmic mRNP granules containing untranslated mRNAs (PubMed:25229427). Shuttles between nucleus and cytoplasm (PubMed:25229427). Localized with DDX1, MBNL1 and TIAL1 in stress granules upon stress (PubMed:18335541). Secreted by mesangial and monocytic cells after inflammatory challenges (PubMed:19483673). {ECO:0000269|PubMed:12604611, ECO:0000269|PubMed:14718551, ECO:0000269|PubMed:18335541, ECO:0000269|PubMed:19483673, ECO:0000269|PubMed:25229427}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}."
1,PPT1,SUBCELLULAR LOCATION: Lysosome {ECO:0000269|PubMed:19941651}. Secreted {ECO:0000250|UniProtKB:P45478}.
2,C6orf120,SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:22340178}. Note=Secreted by hepatocytes.
3,RPL35,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:23636399}.
4,AEBP1,SUBCELLULAR LOCATION: [Isoform 1]: Secreted {ECO:0000250|UniProtKB:Q640N1}.; SUBCELLULAR LOCATION: [Isoform 2]: Cytoplasm {ECO:0000250|UniProtKB:Q640N1}. Nucleus {ECO:0000250|UniProtKB:Q640N1}.
5,ANGPT1,SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:8980223}.
6,PRG2,"SUBCELLULAR LOCATION: [Bone marrow proteoglycan]: Secreted {ECO:0000269|PubMed:25326458, ECO:0000269|PubMed:36213313, ECO:0000269|PubMed:37453717}. Note=The proform is secreted.; SUBCELLULAR LOCATION: [Eosinophil granule major basic protein]: Cytoplasmic vesicle, secretory vesicle. Note=The proform is secreted. The mature protein is found in the matrix of the eosinophil's large specific granule (crystalloid core)."
7,LAMB2,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix, basement membrane. Note=S-laminin is concentrated in the synaptic cleft of the neuromuscular junction."
8,LMAN2L,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane; Single-pass type I membrane protein. Golgi apparatus membrane; Single-pass type I membrane protein. Note=Predominantly found in the endoplasmic reticulum. Partly found in the Golgi.
9,NAGPA,"SUBCELLULAR LOCATION: Golgi apparatus, Golgi stack membrane {ECO:0000269|PubMed:12058031}; Single-pass type I membrane protein {ECO:0000269|PubMed:12058031}. Golgi apparatus, trans-Golgi network {ECO:0000269|PubMed:12058031}. Note=Cis/medial Golgi."
10,PLD3,"SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000269|PubMed:15794758, ECO:0000269|PubMed:22428023}; Single-pass type II membrane protein {ECO:0000269|PubMed:15794758, ECO:0000269|PubMed:22428023}. Lysosome lumen {ECO:0000269|PubMed:29386126, ECO:0000269|PubMed:30312375}. Early endosome membrane {ECO:0000269|PubMed:29386126}; Single-pass type II membrane protein {ECO:0000269|PubMed:15794758, ECO:0000269|PubMed:22428023}. Late endosome membrane {ECO:0000269|PubMed:29386126}; Single-pass type II membrane protein {ECO:0000269|PubMed:15794758, ECO:0000269|PubMed:22428023}. Golgi apparatus membrane {ECO:0000269|PubMed:29368044}; Single-pass type II membrane protein {ECO:0000269|PubMed:29368044}. Endosome membrane {ECO:0000269|PubMed:29368044}; Single-pass type II membrane protein {ECO:0000269|PubMed:29368044}. Note=Localizes to ER-associated vesicles in differentiating myotubes (PubMed:22428023). The soluble form in lysosome arises by proteolytic processing of the membrane-bound form (PubMed:29386126). Colocalizes with APP in endosomes (PubMed:29368044). {ECO:0000269|PubMed:22428023, ECO:0000269|PubMed:29368044, ECO:0000269|PubMed:29386126}.,SUBCELLULAR LOCATION: Early endosome membrane {ECO:0000256|ARBA:ARBA00037858}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00037858}. Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004648}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004648}. Endosome membrane {ECO:0000256|ARBA:ARBA00004639}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004639}. Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}. Late endosome membrane {ECO:0000256|ARBA:ARBA00037797}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00037797}. Lysosome lumen {ECO:0000256|ARBA:ARBA00004227}.,SUBCELLULAR LOCATION: Early endosome membrane {ECO:0000256|ARBA:ARBA00037858}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00037858}. Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004648}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004648}. Endosome membrane {ECO:0000256|ARBA:ARBA00004639}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004639}. Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}. Late endosome membrane {ECO:0000256|ARBA:ARBA00037797}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00037797}. Lysosome lumen {ECO:0000256|ARBA:ARBA00004227}.,SUBCELLULAR LOCATION: Early endosome membrane {ECO:0000256|ARBA:ARBA00037858}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00037858}. Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004648}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004648}. Endosome membrane {ECO:0000256|ARBA:ARBA00004639}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004639}. Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}. Late endosome membrane {ECO:0000256|ARBA:ARBA00037797}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00037797}. Lysosome lumen {ECO:0000256|ARBA:ARBA00004227}.,SUBCELLULAR LOCATION: Early endosome membrane {ECO:0000256|ARBA:ARBA00037858}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00037858}. Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004648}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004648}. Endosome membrane {ECO:0000256|ARBA:ARBA00004639}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004639}. Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}. Late endosome membrane {ECO:0000256|ARBA:ARBA00037797}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00037797}. Lysosome lumen {ECO:0000256|ARBA:ARBA00004227}.,SUBCELLULAR LOCATION: Early endosome membrane {ECO:0000256|ARBA:ARBA00037858}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00037858}. Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004648}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004648}. Endosome membrane {ECO:0000256|ARBA:ARBA00004639}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004639}. Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}. Late endosome membrane {ECO:0000256|ARBA:ARBA00037797}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00037797}. Lysosome lumen {ECO:0000256|ARBA:ARBA00004227}."
11,IFI30,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:10639150, ECO:0000269|PubMed:3136170}. Lysosome {ECO:0000269|PubMed:10639150, ECO:0000269|PubMed:3136170}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|RuleBase:RU369109}. Lysosome {ECO:0000256|RuleBase:RU369109}."
12,MAN2A2,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000250}; Single-pass type II membrane protein {ECO:0000250}.
13,NCLN,"SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000269|PubMed:15257293, ECO:0000269|PubMed:20538592}; Single-pass membrane protein {ECO:0000269|PubMed:15257293, ECO:0000269|PubMed:20538592}."
14,TOR1AIP2,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane; Single-pass membrane protein. Nucleus membrane.
15,SEMA4C,"SUBCELLULAR LOCATION: Postsynaptic density membrane {ECO:0000250}; Single-pass type I membrane protein {ECO:0000250}. Cytoplasmic vesicle, secretory vesicle, synaptic vesicle membrane {ECO:0000250}; Single-pass type I membrane protein {ECO:0000250}."
16,IL17D,SUBCELLULAR LOCATION: Secreted.
17,LAMB1,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix, basement membrane. Note=Major component."
18,STS,"SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle, microneme membrane {ECO:0000269|PubMed:2668275}; Multi-pass membrane protein. Endoplasmic reticulum membrane {ECO:0000269|PubMed:2668275}; Multi-pass membrane protein."
19,RPL28,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:23636399}.
20,MAN2B1,SUBCELLULAR LOCATION: Lysosome.
21,ATP6AP1,"SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000269|PubMed:27231034}; Single-pass type I membrane protein {ECO:0000305}. Endoplasmic reticulum-Golgi intermediate compartment membrane {ECO:0000269|PubMed:27231034}. Cytoplasmic vesicle, secretory vesicle, synaptic vesicle membrane {ECO:0000250|UniProtKB:O54715}; Single-pass type I membrane protein {ECO:0000305}. Cytoplasmic vesicle, clathrin-coated vesicle membrane {ECO:0000250|UniProtKB:O54715}; Single-pass type I membrane protein {ECO:0000305}. Note=Not detected in trans-Golgi network. {ECO:0000269|PubMed:27231034}."
22,CRELD2,SUBCELLULAR LOCATION: Endoplasmic reticulum {ECO:0000269|PubMed:16238698}.
23,BRI3BP,SUBCELLULAR LOCATION: Mitochondrion outer membrane {ECO:0000305|PubMed:17943721}; Multi-pass membrane protein {ECO:0000305|PubMed:17943721}.
24,ADAM15,"SUBCELLULAR LOCATION: Endomembrane system {ECO:0000269|PubMed:12243749}; Single-pass type I membrane protein {ECO:0000269|PubMed:12243749}. Cell junction, adherens junction {ECO:0000269|PubMed:12243749}. Cell projection, cilium, flagellum {ECO:0000250}. Cytoplasmic vesicle, secretory vesicle, acrosome {ECO:0000250}. Note=The majority of the protein is localized in a perinuclear compartment which may correspond to the trans-Golgi network or the late endosome. The pro-protein is the major detectable form on the cell surface, whereas the majority of the protein in the cell is processed (By similarity). {ECO:0000250}."
25,ORM1,SUBCELLULAR LOCATION: Secreted.
26,GBA,SUBCELLULAR LOCATION: Lysosome membrane {ECO:0000256|ARBA:ARBA00004207}; Peripheral membrane protein {ECO:0000256|ARBA:ARBA00004207}; Lumenal side {ECO:0000256|ARBA:ARBA00004207}.
27,LIPA,SUBCELLULAR LOCATION: Lysosome {ECO:0000250|UniProtKB:Q64194}.
28,SCARA3,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000269|PubMed:9580669}; Single-pass type II membrane protein {ECO:0000269|PubMed:9580669}. Golgi apparatus membrane {ECO:0000269|PubMed:9580669}; Single-pass type II membrane protein {ECO:0000269|PubMed:9580669}. Note=Endoplasmic reticulum and/or Golgi.
29,SLC5A3,SUBCELLULAR LOCATION: Apical cell membrane {ECO:0000250|UniProtKB:Q9JKZ2}; Multi-pass membrane protein {ECO:0000255}. Basolateral cell membrane {ECO:0000250|UniProtKB:Q9JKZ2}; Multi-pass membrane protein {ECO:0000255}. Note=Colocalizes with KCNQ1 at the apical membrane of choroid plexus epithelium. {ECO:0000250|UniProtKB:Q9JKZ2}.
30,LTBP1,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:16157329}. Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:16157329, ECO:0000269|PubMed:8617200}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}."
31,PXDN,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:18929642, ECO:0000269|PubMed:19590037, ECO:0000269|PubMed:26178375, ECO:0000269|Ref.9}. Endoplasmic reticulum {ECO:0000269|PubMed:19590037, ECO:0000269|PubMed:25708780}. Cell surface {ECO:0000269|PubMed:25708780}. Secreted, extracellular space, extracellular matrix, basement membrane {ECO:0000250|UniProtKB:Q3UQ28}. Note=Enriched in the peritubular space of fibrotic kidneys. Adheres on the cell surface in 'hot spots' (PubMed:25708780). Only the proteolytically processed PXDN integrates into the extracellular matrix (PubMed:34679700). {ECO:0000269|PubMed:25708780, ECO:0000269|PubMed:34679700}.; SUBCELLULAR LOCATION: [PXDN active fragment]: Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:34679700}."
32,ST8SIA4,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000269|PubMed:9852130}; Single-pass type II membrane protein {ECO:0000305}. Secreted {ECO:0000269|PubMed:9852130}.
33,B3GNT2,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000305}; Single-pass type II membrane protein {ECO:0000305}.
34,HEPACAM2,"SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000269|PubMed:19358830, ECO:0000269|PubMed:22864114}; Single-pass type I membrane protein {ECO:0000255}. Cytoplasm, cytoskeleton, spindle {ECO:0000269|PubMed:19358830, ECO:0000269|PubMed:22864114}. Cytoplasm, cytoskeleton, microtubule organizing center, centrosome {ECO:0000269|PubMed:19358830, ECO:0000269|PubMed:22864114}. Midbody {ECO:0000269|PubMed:22864114}. Note=In interphase, localizes to the Golgi apparatus. Localizes to centrosomes and spindles during prophase, prometaphase, and metaphase of mitosis, and to midbodies at telophase. Translocation to mitotic centrosomes is the result of poly-ADP-ribosylation (PARsylation). {ECO:0000269|PubMed:19358830, ECO:0000269|PubMed:22864114}."
35,NUP210,"SUBCELLULAR LOCATION: Nucleus, nuclear pore complex {ECO:0000269|PubMed:2195063}. Nucleus membrane {ECO:0000269|PubMed:2195063}; Single-pass type I membrane protein {ECO:0000269|PubMed:2195063}. Endoplasmic reticulum membrane {ECO:0000269|PubMed:2195063}; Single-pass type I membrane protein {ECO:0000269|PubMed:2195063}."
36,CTSC,"SUBCELLULAR LOCATION: Lysosome {ECO:0000305|PubMed:7665576}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}."
37,PFN1,"SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton."
38,GALNS,"SUBCELLULAR LOCATION: Lysosome.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}."
39,TUBB2C,"SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}."
40,KNG1,"SUBCELLULAR LOCATION: Secreted, extracellular space.,SUBCELLULAR LOCATION: Secreted, extracellular space {ECO:0000256|ARBA:ARBA00004239}.,SUBCELLULAR LOCATION: Secreted, extracellular space {ECO:0000256|ARBA:ARBA00004239}."
41,MOXD1,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000250}; Single-pass type I membrane protein {ECO:0000250}.
42,WNT5B,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498, ECO:0000256|RuleBase:RU003500}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498, ECO:0000256|RuleBase:RU003500}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498, ECO:0000256|RuleBase:RU003500}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498, ECO:0000256|RuleBase:RU003500}."
43,FKTN,"SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000269|PubMed:25279699, ECO:0000269|PubMed:29477842, ECO:0000305|PubMed:26923585}; Single-pass type II membrane protein {ECO:0000305}. Cytoplasm {ECO:0000250|UniProtKB:Q8R507}. Nucleus {ECO:0000250|UniProtKB:Q8R507}. Note=In retinal tissue, does not localize with the Golgi apparatus. {ECO:0000250|UniProtKB:Q8R507}.,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}.,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}.,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}.,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}.,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}."
44,WNT11,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix."
45,ELOVL4,"SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000255|HAMAP-Rule:MF_03204, ECO:0000269|PubMed:16036915, ECO:0000269|PubMed:20937905}; Multi-pass membrane protein {ECO:0000255|HAMAP-Rule:MF_03204}."
46,POSTN,"SUBCELLULAR LOCATION: Golgi apparatus {ECO:0000250|UniProtKB:Q62009}. Secreted {ECO:0000269|PubMed:18450759, ECO:0000269|PubMed:26273833}. Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:12235007, ECO:0000269|PubMed:18450759}. Note=Colocalizes with BMP1 in the Golgi. {ECO:0000250|UniProtKB:Q62009}."
47,GDF15,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:28572090, ECO:0000269|PubMed:29046435, ECO:0000269|PubMed:31875646, ECO:0000269|PubMed:32694673, ECO:0000269|PubMed:38092039}. Note=Secreted in the plasma. {ECO:0000269|PubMed:28572090, ECO:0000269|PubMed:29046435}."
48,SNRPD2,"SUBCELLULAR LOCATION: Cytoplasm, cytosol {ECO:0000269|PubMed:18984161}. Nucleus {ECO:0000269|PubMed:11991638, ECO:0000269|PubMed:26912367, ECO:0000269|PubMed:28076346, ECO:0000269|PubMed:28502770, ECO:0000269|PubMed:28781166}. Note=SMN-mediated assembly into core snRNPs occurs in the cytosol before SMN-mediated transport to the nucleus to be included in spliceosomes. {ECO:0000305}.,SUBCELLULAR LOCATION: Cytoplasm, cytosol {ECO:0000256|RuleBase:RU365051}. Nucleus {ECO:0000256|ARBA:ARBA00004123, ECO:0000256|RuleBase:RU365051}. Note=SMN-mediated assembly into core snRNPs occurs in the cytosol before SMN-mediated transport to the nucleus to be included in spliceosomes. {ECO:0000256|RuleBase:RU365051}.,SUBCELLULAR LOCATION: Cytoplasm, cytosol {ECO:0000256|RuleBase:RU365051}. Nucleus {ECO:0000256|ARBA:ARBA00004123, ECO:0000256|RuleBase:RU365051}. Note=SMN-mediated assembly into core snRNPs occurs in the cytosol before SMN-mediated transport to the nucleus to be included in spliceosomes. {ECO:0000256|RuleBase:RU365051}."
49,WNT5A,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000305}. Secreted {ECO:0000269|PubMed:26902720}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498, ECO:0000256|RuleBase:RU003500}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498, ECO:0000256|RuleBase:RU003500}."
50,SLC43A3,SUBCELLULAR LOCATION: Basolateral cell membrane {ECO:0000269|PubMed:26455426}; Multi-pass membrane protein {ECO:0000255}.
51,ABI3BP,SUBCELLULAR LOCATION: Secreted {ECO:0000305}.
52,GRN,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:21092856, ECO:0000269|PubMed:26370502}. Lysosome {ECO:0000269|PubMed:21092856, ECO:0000269|PubMed:26370502, ECO:0000269|PubMed:28073925, ECO:0000269|PubMed:28541286, ECO:0000269|PubMed:28743268}. Note=Endocytosed by SORT1 and delivred to lysosomes (PubMed:21092856, PubMed:28073925). Targeted to lysosome by PSAP via M6PR and LRP1, in both biosynthetic and endocytic pathways (PubMed:26370502, PubMed:28073925). Co-localized with GBA1 in the intracellular trafficking compartments until to lysosome (By similarity). {ECO:0000250|UniProtKB:P28798, ECO:0000269|PubMed:21092856, ECO:0000269|PubMed:26370502, ECO:0000269|PubMed:28073925}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
53,EPX,SUBCELLULAR LOCATION: Cytoplasmic granule. Note=Cytoplasmic granules of eosinophils.
54,SERPINI1,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:17040209, ECO:0000269|PubMed:25326458}. Cytoplasmic vesicle, secretory vesicle lumen {ECO:0000305|PubMed:17040209}. Perikaryon {ECO:0000269|PubMed:17040209}."
55,PLD4,"SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000250|UniProtKB:Q8BG07}; Single-pass type II membrane protein {ECO:0000250|UniProtKB:Q8BG07}. Golgi apparatus, trans-Golgi network membrane {ECO:0000250|UniProtKB:Q8BG07}; Single-pass type II membrane protein {ECO:0000250|UniProtKB:Q8BG07}. Nucleus {ECO:0000250|UniProtKB:Q8BG07}. Early endosome {ECO:0000250|UniProtKB:Q8BG07}. Cytoplasmic vesicle, phagosome {ECO:0000250|UniProtKB:Q8BG07}. Note=Activation of microglia induces translocation of PLD4 from the nucleus to the phagosomes. {ECO:0000250|UniProtKB:Q8BG07}."
56,APOB,"SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:22580899}. Secreted {ECO:0000269|PubMed:22580899, ECO:0000269|PubMed:26224785}. Lipid droplet {ECO:0000269|PubMed:28183703}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Lipid droplet {ECO:0000256|ARBA:ARBA00004502}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Lipid droplet {ECO:0000256|ARBA:ARBA00004502}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Lipid droplet {ECO:0000256|ARBA:ARBA00004502}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Lipid droplet {ECO:0000256|ARBA:ARBA00004502}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Lipid droplet {ECO:0000256|ARBA:ARBA00004502}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Lipid droplet {ECO:0000256|ARBA:ARBA00004502}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Lipid droplet {ECO:0000256|ARBA:ARBA00004502}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Lipid droplet {ECO:0000256|ARBA:ARBA00004502}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Lipid droplet {ECO:0000256|ARBA:ARBA00004502}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Lipid droplet {ECO:0000256|ARBA:ARBA00004502}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Lipid droplet {ECO:0000256|ARBA:ARBA00004502}. Secreted {ECO:0000256|ARBA:ARBA00004613}."
57,LMNA,"SUBCELLULAR LOCATION: Nucleus lamina {ECO:0000269|PubMed:2188730, ECO:0000269|PubMed:2344612, ECO:0000269|PubMed:24741066, ECO:0000269|PubMed:37788673, ECO:0000269|PubMed:37832547, ECO:0000305|PubMed:10080180}. Nucleus envelope {ECO:0000269|PubMed:29599122, ECO:0000269|PubMed:37788673, ECO:0000269|PubMed:37832547}. Nucleus, nucleoplasm {ECO:0000269|PubMed:15372542, ECO:0000269|PubMed:24741066, ECO:0000269|PubMed:31548606}. Nucleus matrix {ECO:0000269|PubMed:31548606}. Note=Farnesylation of prelamin-A/C facilitates nuclear envelope targeting and subsequent cleavage by ZMPSTE24/FACE1 to remove the farnesyl group produces mature lamin-A/C, which can then be inserted into the nuclear lamina (PubMed:15317753). EMD is required for proper localization of non-farnesylated prelamin-A/C (PubMed:19323649). Also localizes to the micronuclear envelope in response to response to genome instability (PubMed:37788673). {ECO:0000269|PubMed:15317753, ECO:0000269|PubMed:19323649, ECO:0000269|PubMed:37788673}.; SUBCELLULAR LOCATION: [Isoform C]: Nucleus speckle {ECO:0000269|PubMed:16061563}.,SUBCELLULAR LOCATION: Nucleus envelope {ECO:0000256|ARBA:ARBA00004259}. Nucleus lamina {ECO:0000256|ARBA:ARBA00024186}. Nucleus matrix {ECO:0000256|ARBA:ARBA00004109}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}.,SUBCELLULAR LOCATION: Nucleus envelope {ECO:0000256|ARBA:ARBA00004259}. Nucleus lamina {ECO:0000256|ARBA:ARBA00024186}. Nucleus matrix {ECO:0000256|ARBA:ARBA00004109}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}.,SUBCELLULAR LOCATION: Nucleus envelope {ECO:0000256|ARBA:ARBA00004259}. Nucleus lamina {ECO:0000256|ARBA:ARBA00024186}. Nucleus matrix {ECO:0000256|ARBA:ARBA00004109}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}.,SUBCELLULAR LOCATION: Nucleus envelope {ECO:0000256|ARBA:ARBA00004259}. Nucleus lamina {ECO:0000256|ARBA:ARBA00024186}. Nucleus matrix {ECO:0000256|ARBA:ARBA00004109}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}.,SUBCELLULAR LOCATION: Nucleus envelope {ECO:0000256|ARBA:ARBA00004259}. Nucleus lamina {ECO:0000256|ARBA:ARBA00024186}. Nucleus matrix {ECO:0000256|ARBA:ARBA00004109}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}.,SUBCELLULAR LOCATION: Nucleus envelope {ECO:0000256|ARBA:ARBA00004259}. Nucleus lamina {ECO:0000256|ARBA:ARBA00024186}. Nucleus matrix {ECO:0000256|ARBA:ARBA00004109}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}."
58,FAM20B,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000269|PubMed:19473117}; Single-pass type II membrane protein {ECO:0000269|PubMed:19473117}.
59,CRB2,SUBCELLULAR LOCATION: [Isoform 1]: Apical cell membrane {ECO:0000250|UniProtKB:Q80YA8}; Single-pass type I membrane protein {ECO:0000255}. Cytoplasm {ECO:0000250|UniProtKB:Q80YA8}. Cell junction {ECO:0000250|UniProtKB:Q80YA8}. Note=O-glucosylation is required for localization at the apical plasma membrane (By similarity). Distributed in a complex anisotropic pattern on apical cell edges: the level of CRB2 on a cell edge is inversely correlated with the level of MYH10/myosin-IIB (By similarity). {ECO:0000250|UniProtKB:Q80YA8}.; SUBCELLULAR LOCATION: [Isoform 2]: Secreted {ECO:0000255}.
60,RPL35A,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:23636399}.
61,UBA1,"SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:1376922}. Mitochondrion {ECO:0000269|PubMed:1376922}. Nucleus {ECO:0000269|PubMed:1376922, ECO:0000269|PubMed:22456334}.; SUBCELLULAR LOCATION: [Isoform 1]: Nucleus {ECO:0000269|PubMed:33108101, ECO:0000269|PubMed:7528747, ECO:0000269|PubMed:9099746}.; SUBCELLULAR LOCATION: [Isoform 2]: Cytoplasm {ECO:0000269|PubMed:33108101, ECO:0000269|PubMed:7528747}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}."
62,LZTFL1,"SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:20233871, ECO:0000269|PubMed:22072986}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}."
63,LAMA5,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix, basement membrane. Note=Major component."
64,RPL9,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:23636399}.
65,LTF,"SUBCELLULAR LOCATION: [Isoform 1]: Secreted. Cytoplasmic granule. Note=Secreted into most exocrine fluids by various endothelial cells. Stored in the secondary granules of neutrophils.; SUBCELLULAR LOCATION: [Isoform DeltaLf]: Cytoplasm. Nucleus. Note=Mainly localized in the cytoplasm.,SUBCELLULAR LOCATION: Cytoplasmic granule {ECO:0000256|ARBA:ARBA00004463}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Cytoplasmic granule {ECO:0000256|ARBA:ARBA00004463}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Cytoplasmic granule {ECO:0000256|ARBA:ARBA00004463}. Secreted {ECO:0000256|ARBA:ARBA00004613}."
66,EMILIN1,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:26462740, ECO:0000269|PubMed:31978608}. Note=Found mainly at the interface between amorphous elastin and microfibrils. {ECO:0000303|PubMed:10625608}."
67,COL14A1,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000250|UniProtKB:P32018}."
68,PPT2,SUBCELLULAR LOCATION: Lysosome {ECO:0000269|PubMed:9341199}.
69,MIA3,"SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000269|PubMed:19269366, ECO:0000269|PubMed:21525241, ECO:0000269|PubMed:32101163}; Single-pass membrane protein {ECO:0000269|PubMed:19269366}. Note=Localizes at endoplasmic reticulum exit sites (ERES), also known as transitional endoplasmic reticulum (tER) (PubMed:32101163). SEC16A is required for its proper localization to ERES. After loading of COL7A1 into transport carriers, it is not incorporated into COPII carriers and remains in the endoplasmic reticulum membrane. {ECO:0000269|PubMed:19269366, ECO:0000269|PubMed:21525241, ECO:0000269|PubMed:28442536}."
70,ATRX,"SUBCELLULAR LOCATION: Chromosome, telomere {ECO:0000256|ARBA:ARBA00004574}. Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus. Chromosome, telomere. Nucleus, PML body. Note=Associated with pericentromeric heterochromatin during interphase and mitosis, probably by interacting with CBX5/HP1 alpha. Colocalizes with histone H3.3, DAXX, HIRA and ASF1A at PML-nuclear bodies. Colocalizes with cohesin (SMC1 and SMC3) and MECP2 at the maternal H19 ICR (By similarity). {ECO:0000250}.,SUBCELLULAR LOCATION: Chromosome, telomere {ECO:0000256|ARBA:ARBA00004574}. Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Chromosome, telomere {ECO:0000256|ARBA:ARBA00004574}. Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Chromosome, telomere {ECO:0000256|ARBA:ARBA00004574}. Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Chromosome, telomere {ECO:0000256|ARBA:ARBA00004574}. Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Chromosome, telomere {ECO:0000256|ARBA:ARBA00004574}. Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Chromosome, telomere {ECO:0000256|ARBA:ARBA00004574}. Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Chromosome, telomere {ECO:0000256|ARBA:ARBA00004574}. Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Chromosome, telomere {ECO:0000256|ARBA:ARBA00004574}. Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Chromosome, telomere {ECO:0000256|ARBA:ARBA00004574}. Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Chromosome, telomere {ECO:0000256|ARBA:ARBA00004574}. Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Chromosome, telomere {ECO:0000256|ARBA:ARBA00004574}. Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Chromosome, telomere {ECO:0000256|ARBA:ARBA00004574}. Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}."
71,HGSNAT,"SUBCELLULAR LOCATION: Lysosome membrane {ECO:0000269|PubMed:16960811, ECO:0000269|PubMed:17033958, ECO:0000269|PubMed:17897319}; Multi-pass membrane protein {ECO:0000269|PubMed:16960811, ECO:0000269|PubMed:17033958, ECO:0000269|PubMed:17897319}. Note=Colocalizes with the lysosomal marker LAMP2. The signal peptide is not cleaved upon translocation into the endoplasmic reticulum; the precursor is probably targeted to the lysosomes via the adapter protein complex-mediated pathway that involves tyrosine- and/or dileucine-based conserved amino acid motifs in the last C-terminus 16-amino acid domain."
72,COL11A1,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000255|PROSITE-ProRule:PRU00793}."
73,HSP90B1,"SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000305|PubMed:12475965}. Sarcoplasmic reticulum lumen {ECO:0000250|UniProtKB:P41148}. Melanosome {ECO:0000269|PubMed:12643545, ECO:0000269|PubMed:17081065}. Note=Identified by mass spectrometry in melanosome fractions from stage I to stage IV. {ECO:0000269|PubMed:12643545, ECO:0000269|PubMed:17081065}.,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004319}. Melanosome {ECO:0000256|ARBA:ARBA00004223}. Sarcoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004564}.,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004319}. Melanosome {ECO:0000256|ARBA:ARBA00004223}. Sarcoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004564}.,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004319}. Melanosome {ECO:0000256|ARBA:ARBA00004223}. Sarcoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004564}.,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004319}. Melanosome {ECO:0000256|ARBA:ARBA00004223}. Sarcoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004564}.,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004319}. Melanosome {ECO:0000256|ARBA:ARBA00004223}. Sarcoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004564}.,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004319}. Melanosome {ECO:0000256|ARBA:ARBA00004223}. Sarcoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004564}.,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004319}. Melanosome {ECO:0000256|ARBA:ARBA00004223}. Sarcoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004564}.,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004319}. Melanosome {ECO:0000256|ARBA:ARBA00004223}. Sarcoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004564}.,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004319}. Melanosome {ECO:0000256|ARBA:ARBA00004223}. Sarcoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004564}.,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004319}. Melanosome {ECO:0000256|ARBA:ARBA00004223}. Sarcoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004564}.,SUBCELLULAR LOCATION: Melanosome {ECO:0000256|ARBA:ARBA00004223}."
74,APOH,SUBCELLULAR LOCATION: Secreted.
75,BTD,"SUBCELLULAR LOCATION: Secreted, extracellular space {ECO:0000305|PubMed:9099842, ECO:0000305|PubMed:9654207}."
76,TFPI2,"SUBCELLULAR LOCATION: Secreted.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|PIRNR:PIRNR001620}."
77,VWF,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:10961880}. Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:10961880}. Note=Localized to storage granules.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498, ECO:0000256|PIRNR:PIRNR002495}. Secreted {ECO:0000256|PIRNR:PIRNR002495}. Note=Localized to storage granules. {ECO:0000256|PIRNR:PIRNR002495}."
78,NXPH4,SUBCELLULAR LOCATION: Secreted {ECO:0000305}.
79,PLAU,SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:24434139}.
80,FBN2,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:32329225}. Note=Fibrillin-2 and Placensin chains are still linked together during the secretion from cells, but are subsequently separated by furin. {ECO:0000269|PubMed:32329225}.; SUBCELLULAR LOCATION: [Fibrillin-2]: Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:8120105}.; SUBCELLULAR LOCATION: [Placensin]: Secreted {ECO:0000269|PubMed:32329225}. Note=Secreted by placental cells. {ECO:0000269|PubMed:32329225}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}."
81,PCBP2,"SUBCELLULAR LOCATION: Nucleus {ECO:0000269|PubMed:7607214}. Cytoplasm {ECO:0000269|PubMed:19029303, ECO:0000269|PubMed:7607214}. Note=Loosely bound in the nucleus (PubMed:7607214). May shuttle between the nucleus and the cytoplasm (PubMed:7607214). {ECO:0000269|PubMed:7607214}."
82,CFH,"SUBCELLULAR LOCATION: Secreted.; SUBCELLULAR LOCATION: Note=(Microbial infection) In the mosquito midgut, localizes to P.falciparum (NF54 strain) macrogamete and young zygote cell membranes. {ECO:0000269|PubMed:23332154}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
83,FKBP9,SUBCELLULAR LOCATION: Endoplasmic reticulum {ECO:0000255|PROSITE-ProRule:PRU10138}.
84,METTL9,"SUBCELLULAR LOCATION: Endoplasmic reticulum {ECO:0000269|PubMed:34562450, ECO:0000305|PubMed:33563959}. Mitochondrion {ECO:0000305|PubMed:33563959}. Note=Colocalizes with membranous compartments such as the endoplasmic reticulum and mitochondria. {ECO:0000269|PubMed:33563959}."
85,HNRNPU,"SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000269|PubMed:10671544, ECO:0000269|PubMed:11003645, ECO:0000269|PubMed:1324173, ECO:0000269|PubMed:14608463, ECO:0000269|PubMed:15563465, ECO:0000269|PubMed:19617346, ECO:0000269|PubMed:21242313, ECO:0000269|PubMed:8174554, ECO:0000269|PubMed:9353307, ECO:0000269|PubMed:9405365}. Nucleus matrix {ECO:0000269|PubMed:1324173, ECO:0000269|PubMed:14608463, ECO:0000269|PubMed:8174554}. Chromosome {ECO:0000269|PubMed:11003645, ECO:0000269|PubMed:14608463, ECO:0000269|PubMed:15563465}. Nucleus speckle {ECO:0000269|PubMed:9353307}. Cytoplasm, cytoskeleton, microtubule organizing center, centrosome {ECO:0000269|PubMed:14654843, ECO:0000269|PubMed:25986610}. Chromosome, centromere, kinetochore {ECO:0000269|PubMed:21242313}. Cytoplasm, cytoskeleton, spindle {ECO:0000269|PubMed:21242313, ECO:0000269|PubMed:25986610}. Cytoplasm, cytoskeleton, spindle pole {ECO:0000269|PubMed:21242313}. Midbody {ECO:0000269|PubMed:25986610}. Cytoplasm {ECO:0000269|PubMed:19029303}. Cell surface {ECO:0000269|PubMed:7993898}. Cytoplasmic granule {ECO:0000269|PubMed:17289661}. Note=Localizes at inactive X chromosome (Xi) regions (PubMed:11003645, PubMed:14608463, PubMed:15563465). Localizes in the nucleus during interphase (PubMed:21242313). At metaphase, localizes with mitotic spindle microtubules (MTs) (PubMed:21242313). At anaphase, localizes in the mitotic spindle midzone (PubMed:21242313). Localizes in spindle MTs proximal to spindle poles in a TPX2- and AURKA-dependent manner (PubMed:21242313). The Ser-59 phosphorylated form localizes to centrosomes during prophase and metaphase, to mitotic spindles in anaphase and to the midbody during cytokinesis (PubMed:25986610). Colocalizes with SMARCA4 in the nucleus (By similarity). Colocalizes with CBX5 in the nucleus (PubMed:19617346). Colocalizes with NR3C1 in nuclear speckles (PubMed:9353307). Localized in cytoplasmic ribonucleoprotein (RNP) granules containing untranslated mRNAs (PubMed:17289661). {ECO:0000250|UniProtKB:Q8VEK3, ECO:0000269|PubMed:11003645, ECO:0000269|PubMed:14608463, ECO:0000269|PubMed:15563465, ECO:0000269|PubMed:17289661, ECO:0000269|PubMed:19617346, ECO:0000269|PubMed:21242313, ECO:0000269|PubMed:25986610, ECO:0000269|PubMed:9353307}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}."
86,ATF6B,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000269|PubMed:11256944}; Single-pass type II membrane protein {ECO:0000255}.; SUBCELLULAR LOCATION: [Processed cyclic AMP-dependent transcription factor ATF-6 beta]: Nucleus {ECO:0000269|PubMed:11256944}. Note=Under ER stress the cleaved N-terminal cytoplasmic domain translocates into the nucleus. {ECO:0000269|PubMed:11256944}.
87,NPTX1,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:34788392}. Cytoplasmic vesicle, secretory vesicle {ECO:0000269|PubMed:34788392, ECO:0000305}. Endoplasmic reticulum {ECO:0000269|PubMed:34788392, ECO:0000305}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|RuleBase:RU362112}."
88,C4BPB,SUBCELLULAR LOCATION: Secreted.
89,SV2A,"SUBCELLULAR LOCATION: Presynapse {ECO:0000250|UniProtKB:Q9JIS5}. Cytoplasmic vesicle, secretory vesicle, synaptic vesicle membrane {ECO:0000250|UniProtKB:Q02563}; Multi-pass membrane protein {ECO:0000250|UniProtKB:Q02563}. Note=Enriched in chromaffin granules, not present in adrenal microsomes. Associated with both insulin granules and synaptic-like microvesicles in insulin-secreting cells of the pancreas (By similarity). Colocalizes with ATP2B1 at photoreceptor synaptic terminals. {ECO:0000250|UniProtKB:Q02563, ECO:0000250|UniProtKB:Q9JIS5}."
90,TUBB,"SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000269|PubMed:26637975}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}."
91,PPIB,SUBCELLULAR LOCATION: Virion {ECO:0000269|PubMed:20147391}. Note=(Microbial infection).; SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000269|PubMed:1530944}. Melanosome {ECO:0000269|PubMed:17081065}. Note=Identified by mass spectrometry in melanosome fractions from stage I to stage IV (PubMed:17081065). {ECO:0000269|PubMed:17081065}.
92,ITPR3,"SUBCELLULAR LOCATION: Endoplasmic reticulum membrane; Multi-pass membrane protein.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|RuleBase:RU368044}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU368044}."
93,ANGPTL4,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:10698685, ECO:0000269|PubMed:14583458, ECO:0000269|PubMed:17068295, ECO:0000269|PubMed:19270337, ECO:0000269|PubMed:21398697, ECO:0000269|PubMed:29899519}. Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:17068295, ECO:0000269|PubMed:21398697}. Note=The unprocessed form interacts with the extracellular matrix (PubMed:17068295, PubMed:21398697). This may constitute a dynamic reservoir, a regulatory mechanism of the bioavailability of ANGPTL4 (Probable). {ECO:0000269|PubMed:17068295, ECO:0000269|PubMed:21398697, ECO:0000305|PubMed:17068295}."
94,CHGA,"SUBCELLULAR LOCATION: [Serpinin]: Secreted {ECO:0000250|UniProtKB:P26339}. Cytoplasmic vesicle, secretory vesicle {ECO:0000250|UniProtKB:P26339}. Note=Pyroglutaminated serpinin localizes to secretory vesicle. {ECO:0000250|UniProtKB:P26339}.; SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle {ECO:0000250|UniProtKB:P10354}. Cytoplasmic vesicle, secretory vesicle, neuronal dense core vesicle {ECO:0000250|UniProtKB:P10354}. Secreted {ECO:0000269|PubMed:25326458, ECO:0000269|PubMed:37453717}. Note=Associated with the secretory granule membrane through direct interaction to SCG3 that in turn binds to cholesterol-enriched lipid rafts in intragranular conditions. In pituitary gonadotropes, located in large secretory granules. {ECO:0000250|UniProtKB:P10354}.,SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle {ECO:0000256|ARBA:ARBA00004398}. Secreted {ECO:0000256|ARBA:ARBA00004613}. Vesicle {ECO:0000256|ARBA:ARBA00004373}."
95,CD46,"SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle, acrosome inner membrane {ECO:0000269|PubMed:12112588, ECO:0000269|PubMed:14597734, ECO:0000269|PubMed:15307194}; Single-pass type I membrane protein {ECO:0000269|PubMed:12112588, ECO:0000269|PubMed:14597734, ECO:0000269|PubMed:15307194}. Note=Inner acrosomal membrane of spermatozoa. Internalized upon binding of Measles virus, Herpesvirus 6 or Neisseria gonorrhoeae, which results in an increased susceptibility of infected cells to complement-mediated injury. In cancer cells or cells infected by Neisseria, shedding leads to a soluble peptide.,SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle, acrosome inner membrane {ECO:0000256|ARBA:ARBA00004537}; Single-pass type I membrane protein {ECO:0000256|ARBA:ARBA00004537}.,SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle, acrosome inner membrane {ECO:0000256|ARBA:ARBA00004537}; Single-pass type I membrane protein {ECO:0000256|ARBA:ARBA00004537}.,SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle, acrosome inner membrane {ECO:0000256|ARBA:ARBA00004537}; Single-pass type I membrane protein {ECO:0000256|ARBA:ARBA00004537}.,SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle, acrosome inner membrane {ECO:0000256|ARBA:ARBA00004537}; Single-pass type I membrane protein {ECO:0000256|ARBA:ARBA00004537}.,SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle, acrosome inner membrane {ECO:0000256|ARBA:ARBA00004537}; Single-pass type I membrane protein {ECO:0000256|ARBA:ARBA00004537}.,SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle, acrosome inner membrane {ECO:0000256|ARBA:ARBA00004537}; Single-pass type I membrane protein {ECO:0000256|ARBA:ARBA00004537}."
96,OGFOD1,SUBCELLULAR LOCATION: [Isoform 1]: Cytoplasm. Nucleus. Note=Mainly nuclear. A portion relocalizes to cytoplasmic stress granules upon stress.; SUBCELLULAR LOCATION: [Isoform 2]: Cytoplasm.
97,DCN,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix. Secreted {ECO:0000269|PubMed:25326458, ECO:0000269|PubMed:36213313, ECO:0000269|PubMed:37453717}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|RuleBase:RU364097}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|RuleBase:RU364097}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|RuleBase:RU364097}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|RuleBase:RU364097}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|RuleBase:RU364097}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|RuleBase:RU364097}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498, ECO:0000256|PIRNR:PIRNR002490, ECO:0000256|RuleBase:RU364097}."
98,GDPD5,"SUBCELLULAR LOCATION: Endomembrane system {ECO:0000250|UniProtKB:Q640M6}; Multi-pass membrane protein {ECO:0000255}. Cytoplasm, perinuclear region {ECO:0000250|UniProtKB:Q640M6}. Cell projection, growth cone {ECO:0000250|UniProtKB:Q640M6}. Note=In a punctate perinuclear pattern. {ECO:0000250|UniProtKB:Q640M6}."
99,LAMC2,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix, basement membrane. Note=Major component."
100,F5,SUBCELLULAR LOCATION: Secreted {ECO:0000250}.
101,PRELP,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix."
102,ASAH1,"SUBCELLULAR LOCATION: Lysosome {ECO:0000269|PubMed:12764132}. Secreted {ECO:0000269|PubMed:7744740}. Note=Secretion is extremely low and localization to lysosomes is mannose-6-phosphate receptor-dependent. {ECO:0000269|PubMed:11451951}.; SUBCELLULAR LOCATION: [Isoform 2]: Nucleus {ECO:0000305|PubMed:22927646}. Cytoplasm {ECO:0000305|PubMed:22927646}. Note=A localization to the nucleus and the cytoplasm has also been reported for ASAH1, most probably for isoforms devoid of a signal peptide. {ECO:0000305|PubMed:22927646}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371, ECO:0000256|PIRNR:PIRNR017632}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371, ECO:0000256|PIRNR:PIRNR017632}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371, ECO:0000256|PIRNR:PIRNR017632}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371, ECO:0000256|PIRNR:PIRNR017632}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371, ECO:0000256|PIRNR:PIRNR017632}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371, ECO:0000256|PIRNR:PIRNR017632}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371, ECO:0000256|PIRNR:PIRNR017632}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371, ECO:0000256|PIRNR:PIRNR017632}. Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371, ECO:0000256|PIRNR:PIRNR017632}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}."
103,QSOX1,"SUBCELLULAR LOCATION: [Isoform 1]: Golgi apparatus membrane {ECO:0000269|PubMed:17331072, ECO:0000269|PubMed:23704371}; Single-pass membrane protein {ECO:0000305|PubMed:17331072}. Secreted {ECO:0000269|PubMed:29757379}. Note=A small proportion is secreted, probably via a proteolytic cleavage that removes the membrane anchor. {ECO:0000305|PubMed:29757379}.; SUBCELLULAR LOCATION: [Isoform 2]: Secreted {ECO:0000269|PubMed:10708601}. Note=Found in the extracellular medium of quiescent cells but is not found in proliferating cells. {ECO:0000269|PubMed:10708601}."
104,ISLR,SUBCELLULAR LOCATION: Secreted {ECO:0000305}.
105,GMPS,"SUBCELLULAR LOCATION: Cytoplasm, cytosol {ECO:0000269|PubMed:8089153}."
106,TGFB1,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: [Latency-associated peptide]: Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:17827158}.; SUBCELLULAR LOCATION: [Transforming growth factor beta-1]: Secreted {ECO:0000269|PubMed:17827158, ECO:0000269|PubMed:29483653}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}."
107,PLOD3,"SUBCELLULAR LOCATION: Rough endoplasmic reticulum {ECO:0000269|PubMed:10934207}. Endoplasmic reticulum lumen {ECO:0000269|PubMed:20470363}. Endoplasmic reticulum membrane {ECO:0000250|UniProtKB:Q9R0E1}; Peripheral membrane protein {ECO:0000250|UniProtKB:Q9R0E1}; Lumenal side {ECO:0000250|UniProtKB:Q9R0E1}. Secreted {ECO:0000269|PubMed:21465473}. Secreted, extracellular space {ECO:0000250|UniProtKB:Q9R0E1}. Note=The majority of the secreted protein is associated with the extracellular matrix. {ECO:0000250|UniProtKB:Q9R0E1}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004367}; Peripheral membrane protein {ECO:0000256|ARBA:ARBA00004367}; Lumenal side {ECO:0000256|ARBA:ARBA00004367}."
108,GMPPB,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:23768512}.
109,TMEM9,"SUBCELLULAR LOCATION: Lysosome membrane {ECO:0000269|PubMed:12359240, ECO:0000305|PubMed:30374053}; Single-pass type I membrane protein {ECO:0000255}. Late endosome membrane {ECO:0000269|PubMed:12359240, ECO:0000269|PubMed:30374053}; Single-pass type I membrane protein {ECO:0000255}. Endosome, multivesicular body membrane {ECO:0000269|PubMed:30374053}; Single-pass type I membrane protein {ECO:0000255}."
110,GLB1,"SUBCELLULAR LOCATION: [Isoform 1]: Lysosome {ECO:0000269|PubMed:2511208, ECO:0000269|PubMed:3084261}.; SUBCELLULAR LOCATION: [Isoform 2]: Cytoplasm, perinuclear region {ECO:0000269|PubMed:2511208}. Note=Localized to the perinuclear area of the cytoplasm but not to lysosomes. {ECO:0000269|PubMed:2511208}."
111,APOD,"SUBCELLULAR LOCATION: Secreted.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
112,ITIH2,"SUBCELLULAR LOCATION: Secreted.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
113,GAPDHS,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000250}.
114,LGMN,SUBCELLULAR LOCATION: Lysosome {ECO:0000269|PubMed:9872320}.
115,IGLON5,SUBCELLULAR LOCATION: Secreted {ECO:0000305}.
116,CASD1,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000269|PubMed:26169044}; Multi-pass membrane protein {ECO:0000305}.
117,SERPINB2,"SUBCELLULAR LOCATION: Cytoplasm. Secreted, extracellular space."
118,SERPINF1,SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:17081065}. Melanosome {ECO:0000269|PubMed:17081065}. Note=Enriched in stage I melanosomes.
119,NXF2,"SUBCELLULAR LOCATION: Nucleus, nucleoplasm. Cytoplasm. Note=Localized in the nucleoplasm and at the nuclear envelope. Shuttles between the nucleus and the cytoplasm."
120,OGN,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000250|UniProtKB:Q8MJF1}."
121,DMXL2,"SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle, synaptic vesicle membrane {ECO:0000269|PubMed:11809763}; Peripheral membrane protein {ECO:0000269|PubMed:11809763}. Cytoplasmic vesicle, secretory vesicle, neuronal dense core vesicle {ECO:0000250|UniProtKB:Q8BPN8}. Note=The external layer of the inferior boundary for the hypothalamus part of the human brain (the so called median eminence (ME)) displayed a punctate pattern of expression; expression also observed in the cell bodies lining the third ventricle, in the long processes extending from these cell bodies toward the external layer of the ME, in small clear vesicles, and in large dense core vesicles. {ECO:0000250|UniProtKB:Q8BPN8}."
122,FN1,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000305|PubMed:29100092}. Secreted {ECO:0000250|UniProtKB:P11276}."
123,BDNF,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:11152678, ECO:0000269|PubMed:19467646, ECO:0000269|PubMed:8527932}.; SUBCELLULAR LOCATION: [BDNF precursor form]: Secreted {ECO:0000269|PubMed:11152678, ECO:0000269|PubMed:19467646}. Note=A proportion of BDNF is secreted as immature precursor (proBDNF). {ECO:0000269|PubMed:11152678, ECO:0000269|PubMed:19467646}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|RuleBase:RU364086}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|RuleBase:RU364086}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|RuleBase:RU364086}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|RuleBase:RU364086}."
124,PRSS23,SUBCELLULAR LOCATION: Secreted {ECO:0000305}.
125,PLBD1,SUBCELLULAR LOCATION: Lysosome {ECO:0000250}.
126,RCN3,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000269|PubMed:16433634}.
127,RPL18,"SUBCELLULAR LOCATION: Cytoplasm, cytosol {ECO:0000269|PubMed:25957688}. Cytoplasm {ECO:0000305|PubMed:23636399, ECO:0000305|PubMed:25901680}. Rough endoplasmic reticulum {ECO:0000250|UniProtKB:Q95342}. Note=Detected on cytosolic polysomes (PubMed:25957688). Detected in ribosomes that are associated with the rough endoplasmic reticulum (By similarity). {ECO:0000250|UniProtKB:Q95342, ECO:0000269|PubMed:25957688}.,SUBCELLULAR LOCATION: Rough endoplasmic reticulum {ECO:0000256|ARBA:ARBA00004427}.,SUBCELLULAR LOCATION: Rough endoplasmic reticulum {ECO:0000256|ARBA:ARBA00004427}.,SUBCELLULAR LOCATION: Rough endoplasmic reticulum {ECO:0000256|ARBA:ARBA00004427}.,SUBCELLULAR LOCATION: Rough endoplasmic reticulum {ECO:0000256|ARBA:ARBA00004427}.,SUBCELLULAR LOCATION: Rough endoplasmic reticulum {ECO:0000256|ARBA:ARBA00004427}.,SUBCELLULAR LOCATION: Rough endoplasmic reticulum {ECO:0000256|ARBA:ARBA00004427}.,SUBCELLULAR LOCATION: Rough endoplasmic reticulum {ECO:0000256|ARBA:ARBA00004427}."
128,CCDC80,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000250}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}."
129,ELANE,"SUBCELLULAR LOCATION: Cytoplasmic vesicle, phagosome {ECO:0000269|PubMed:10947984}. Note=Localized in phagolysosomes following ingestion of E.coli by neutrophils. {ECO:0000269|PubMed:10947984}."
130,HEXA,"SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}."
131,ABCC5,"SUBCELLULAR LOCATION: Basolateral cell membrane {ECO:0000269|PubMed:24836561}; Multi-pass membrane protein {ECO:0000255}. Golgi apparatus lumen {ECO:0000269|PubMed:24836561}. Endosome membrane {ECO:0000269|PubMed:24836561}. Cytoplasmic granule {ECO:0000250|UniProtKB:Q9R1X5}. Apical cell membrane {ECO:0000269|PubMed:15501592}; Multi-pass membrane protein {ECO:0000255}. Note=In most cells, routes to the basolateral plasma membrane, but in the brain capillary endothelial cells that form the blood-brain barrier, resides in the apical membrane. {ECO:0000269|PubMed:15501592, ECO:0000269|PubMed:24836561}."
132,PON1,"SUBCELLULAR LOCATION: Secreted, extracellular space."
133,HP,SUBCELLULAR LOCATION: Secreted.
134,IGF2R,"SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000269|PubMed:18817523}; Single-pass type I membrane protein {ECO:0000269|PubMed:2957598}. Endosome membrane {ECO:0000269|PubMed:18817523}; Single-pass type I membrane protein {ECO:0000269|PubMed:2957598}. Note=Mainly localized in the Golgi at steady state and not detectable in lysosome (PubMed:18817523). Colocalized with DPP4 in internalized cytoplasmic vesicles adjacent to the cell surface (PubMed:10900005). {ECO:0000269|PubMed:10900005, ECO:0000269|PubMed:18817523}."
135,SULF2,"SUBCELLULAR LOCATION: Endoplasmic reticulum {ECO:0000250|UniProtKB:Q8VI60}. Golgi apparatus, Golgi stack {ECO:0000250|UniProtKB:Q8VI60}. Cell surface {ECO:0000269|PubMed:12368295, ECO:0000269|PubMed:35294879}.; SUBCELLULAR LOCATION: [Extracellular sulfatase Sulf-2 secreted form]: Secreted {ECO:0000269|PubMed:12368295, ECO:0000269|PubMed:30788513, ECO:0000269|PubMed:35294879}."
136,RPL21,"SUBCELLULAR LOCATION: Cytoplasm, cytosol {ECO:0000269|PubMed:25957688}. Cytoplasm {ECO:0000305|PubMed:23636399, ECO:0000305|PubMed:25901680}. Endoplasmic reticulum {ECO:0000250|UniProtKB:P49666}. Note=Detected on cytosolic polysomes (PubMed:25957688). Detected in ribosomes that are associated with the rough endoplasmic reticulum (By similarity). {ECO:0000250|UniProtKB:P49666, ECO:0000269|PubMed:25957688}.,SUBCELLULAR LOCATION: Cytoplasm, cytosol {ECO:0000256|ARBA:ARBA00004514}.,SUBCELLULAR LOCATION: Cytoplasm, cytosol {ECO:0000256|ARBA:ARBA00004514}."
137,TIMP1,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:1730286, ECO:0000269|PubMed:24635319, ECO:0000269|PubMed:3010309, ECO:0000269|PubMed:3839290, ECO:0000269|PubMed:3903517, ECO:0000269|PubMed:8541540}."
138,YWHAQ,"SUBCELLULAR LOCATION: Cytoplasm. Note=In neurons, axonally transported to the nerve terminals."
139,CCDC126,"SUBCELLULAR LOCATION: Secreted {ECO:0000305}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
140,TOR1B,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000269|PubMed:15147511}. Nucleus membrane {ECO:0000269|PubMed:15147511}.
141,PTGFRN,"SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000250}; Single-pass type I membrane protein {ECO:0000250}. Golgi apparatus, trans-Golgi network membrane {ECO:0000250}; Single-pass type I membrane protein {ECO:0000250}."
142,TPP1,SUBCELLULAR LOCATION: Lysosome {ECO:0000269|PubMed:19941651}. Melanosome {ECO:0000269|PubMed:12643545}. Note=Identified by mass spectrometry in melanosome fractions from stage I to stage IV. {ECO:0000269|PubMed:12643545}.
143,COL6A3,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000250}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}."
144,HSPA13,SUBCELLULAR LOCATION: Microsome. Endoplasmic reticulum.
145,ABCC4,"SUBCELLULAR LOCATION: Basolateral cell membrane {ECO:0000269|PubMed:11106685, ECO:0000269|PubMed:12883481, ECO:0000269|PubMed:26721430}; Multi-pass membrane protein {ECO:0000255}. Apical cell membrane {ECO:0000269|PubMed:11856762, ECO:0000269|PubMed:26721430}; Multi-pass membrane protein {ECO:0000255}. Note=Its localization to the basolateral or apical membranes is tissue-dependent. {ECO:0000305|PubMed:26721430}."
146,ACTN4,"SUBCELLULAR LOCATION: Nucleus {ECO:0000269|PubMed:22351778, ECO:0000269|PubMed:9508771}. Cytoplasm {ECO:0000269|PubMed:22351778, ECO:0000269|PubMed:9508771}. Cell junction {ECO:0000250|UniProtKB:P57780}. Cytoplasm, cytoskeleton, stress fiber {ECO:0000269|PubMed:9508771}. Cytoplasm, perinuclear region {ECO:0000250|UniProtKB:P57780}. Note=Localized in cytoplasmic mRNP granules containing untranslated mRNAs. Expressed in the perinuclear rim and manchette structure in early elongating spermatids during spermiogenesis (By similarity). Nuclear translocation can be induced by the PI3 kinase inhibitor wortmannin or by cytochalasin D. Exclusively localized in the nucleus in a limited number of cell lines (breast cancer cell line MCF-7, oral floor cancer IMC-2, and bladder cancer KU-7). {ECO:0000250|UniProtKB:P57780, ECO:0000269|PubMed:17289661, ECO:0000269|PubMed:9508771}.,SUBCELLULAR LOCATION: Cell junction {ECO:0000256|ARBA:ARBA00004282}. Cytoplasm, cytoskeleton, stress fiber {ECO:0000256|ARBA:ARBA00004529}. Cytoplasm, perinuclear region {ECO:0000256|ARBA:ARBA00004556}.,SUBCELLULAR LOCATION: Cell junction {ECO:0000256|ARBA:ARBA00004282}. Cytoplasm, cytoskeleton, stress fiber {ECO:0000256|ARBA:ARBA00004529}. Cytoplasm, perinuclear region {ECO:0000256|ARBA:ARBA00004556}.,SUBCELLULAR LOCATION: Cell junction {ECO:0000256|ARBA:ARBA00004282}. Cytoplasm, cytoskeleton, stress fiber {ECO:0000256|ARBA:ARBA00004529}. Cytoplasm, perinuclear region {ECO:0000256|ARBA:ARBA00004556}.,SUBCELLULAR LOCATION: Cell junction {ECO:0000256|ARBA:ARBA00004282}. Cytoplasm, perinuclear region {ECO:0000256|ARBA:ARBA00004556}.,SUBCELLULAR LOCATION: Cell junction {ECO:0000256|ARBA:ARBA00004282}. Cytoplasm, perinuclear region {ECO:0000256|ARBA:ARBA00004556}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton, stress fiber {ECO:0000256|ARBA:ARBA00004529}.,SUBCELLULAR LOCATION: Cell junction {ECO:0000256|ARBA:ARBA00004282}. Cytoplasm, perinuclear region {ECO:0000256|ARBA:ARBA00004556}."
147,RNF13,"SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000269|PubMed:18794910, ECO:0000269|PubMed:23378536, ECO:0000269|PubMed:24387786}; Single-pass type I membrane protein {ECO:0000255}. Late endosome membrane {ECO:0000305|PubMed:24387786}; Single-pass type I membrane protein {ECO:0000255}. Lysosome membrane {ECO:0000269|PubMed:17897319}; Single-pass type I membrane protein {ECO:0000255}. Nucleus inner membrane {ECO:0000250|UniProtKB:O54965}; Single-pass type I membrane protein {ECO:0000255}. Note=Under certain conditions, relocalizes to recycling endosomes and to the inner nuclear membrane. {ECO:0000250|UniProtKB:O54965}."
148,FGL2,SUBCELLULAR LOCATION: Secreted.
149,COL6A1,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000250}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}."
150,PCBP3,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:14612387}.
151,DKK3,"SUBCELLULAR LOCATION: Secreted.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
152,FGB,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:10074346, ECO:0000269|PubMed:19296670, ECO:0000269|PubMed:9628725}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
153,BCHE,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:19368529, ECO:0000269|PubMed:19542320}."
154,RSL1D1,"SUBCELLULAR LOCATION: Nucleus, nucleolus {ECO:0000269|PubMed:11790298, ECO:0000269|PubMed:12429849, ECO:0000269|PubMed:18678645, ECO:0000269|PubMed:22419112}. Note=Colocalizes with ING1 in the nucleolus after UV stress. {ECO:0000269|PubMed:22419112}."
155,NAGA,SUBCELLULAR LOCATION: Lysosome {ECO:0000269|PubMed:9741689}.
156,LEMD2,"SUBCELLULAR LOCATION: Nucleus inner membrane {ECO:0000269|PubMed:16339967, ECO:0000269|PubMed:17097643, ECO:0000269|PubMed:28242692, ECO:0000269|PubMed:30905398, ECO:0000269|PubMed:32494070}; Multi-pass membrane protein {ECO:0000269|PubMed:16339967}. Nucleus envelope {ECO:0000269|PubMed:28242692}. Cytoplasm, cytoskeleton, spindle {ECO:0000269|PubMed:32494070}. Note=Lamina-associated protein residing in the inner nuclear membrane (INM) of the nuclear envelope (NE) (PubMed:16339967). The localization to the INM is dependent on LMNA (PubMed:16339967). Evenly distributed around the NE during interphase (PubMed:16339967). During metaphase, found in a reticular network (PubMed:28242692). Recruited to the reforming NE on chromatin disks in early anaphase (PubMed:28242692). In late anaphase, concentrates at the NE core proximal to spindle microtubules, and then broadening to a distributed nuclear rim pattern (PubMed:28242692, PubMed:32494070). {ECO:0000269|PubMed:16339967, ECO:0000269|PubMed:28242692, ECO:0000269|PubMed:32494070}."
157,CHSY3,"SUBCELLULAR LOCATION: Golgi apparatus, Golgi stack membrane {ECO:0000305|PubMed:12907687}; Single-pass type II membrane protein {ECO:0000305|PubMed:12907687}."
158,SERPINA1,"SUBCELLULAR LOCATION: Secreted. Endoplasmic reticulum. Note=The S and Z allele are not secreted effectively and accumulate intracellularly in the endoplasmic reticulum.; SUBCELLULAR LOCATION: [Short peptide from AAT]: Secreted, extracellular space, extracellular matrix."
159,GALNT5,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000250}; Single-pass type II membrane protein {ECO:0000250}.
160,ADAMTS15,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000250|UniProtKB:P59384}. Cell surface {ECO:0000250|UniProtKB:P59384}."
161,POFUT2,"SUBCELLULAR LOCATION: Endoplasmic reticulum {ECO:0000269|PubMed:15233996}. Golgi apparatus {ECO:0000269|PubMed:15233996}. Note=Mainly located in the endoplasmic reticulum. {ECO:0000269|PubMed:15233996}.,SUBCELLULAR LOCATION: Endoplasmic reticulum {ECO:0000256|ARBA:ARBA00004240}.,SUBCELLULAR LOCATION: Endoplasmic reticulum {ECO:0000256|ARBA:ARBA00004240}.,SUBCELLULAR LOCATION: Endoplasmic reticulum {ECO:0000256|ARBA:ARBA00004240}.,SUBCELLULAR LOCATION: Endoplasmic reticulum {ECO:0000256|ARBA:ARBA00004240}."
162,HSD11B1,"SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000269|PubMed:10497248, ECO:0000269|PubMed:15280030}; Single-pass type II membrane protein {ECO:0000269|PubMed:10497248}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004648}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004648}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004648}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004648}."
163,COL3A1,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000255|PROSITE-ProRule:PRU00793}."
164,TOR3A,"SUBCELLULAR LOCATION: Cytoplasm. Endoplasmic reticulum lumen.,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004319}."
165,HMCN1,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix, basement membrane {ECO:0000250|UniProtKB:D3YXG0}. Cytoplasm {ECO:0000250|UniProtKB:D3YXG0}. Cell junction {ECO:0000250|UniProtKB:D3YXG0}. Cleavage furrow {ECO:0000250|UniProtKB:D3YXG0}. Note=Has been detected in the glomerular basement membrane in one study. However, another study found expression in the glomerular mesangial matrix but not in the glomerular basement membrane. The antibody used to determine subcellular location does not distinguish between HMCN1 and HMCN2. {ECO:0000250|UniProtKB:D3YXG0}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}."
166,C4BPA,SUBCELLULAR LOCATION: Secreted.
167,CALR,"SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000269|PubMed:10358038, ECO:0000269|PubMed:11149926}. Cytoplasm, cytosol {ECO:0000269|PubMed:11149926}. Secreted, extracellular space, extracellular matrix {ECO:0000305}. Cell surface {ECO:0000269|PubMed:10358038}. Sarcoplasmic reticulum lumen {ECO:0000250|UniProtKB:P28491}. Cytoplasmic vesicle, secretory vesicle, Cortical granule {ECO:0000250|UniProtKB:Q8K3H7}. Cytolytic granule {ECO:0000269|PubMed:8418194}. Note=Also found in cell surface (T cells), cytosol and extracellular matrix (PubMed:10358038). During oocyte maturation and after parthenogenetic activation accumulates in cortical granules. In pronuclear and early cleaved embryos localizes weakly to cytoplasm around nucleus and more strongly in the region near the cortex (By similarity). In cortical granules of non-activated oocytes, is exocytosed during the cortical reaction in response to oocyte activation (By similarity). {ECO:0000250|UniProtKB:P28491, ECO:0000250|UniProtKB:Q8K3H7, ECO:0000269|PubMed:8418194}.,SUBCELLULAR LOCATION: Cell surface {ECO:0000256|ARBA:ARBA00004241}. Cytoplasm, cytosol {ECO:0000256|ARBA:ARBA00004514}. Cytoplasmic vesicle, secretory vesicle, Cortical granule {ECO:0000256|ARBA:ARBA00037865}. Sarcoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004564}. Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Cell surface {ECO:0000256|ARBA:ARBA00004241}. Cytoplasm, cytosol {ECO:0000256|ARBA:ARBA00004514}. Cytoplasmic vesicle, secretory vesicle, Cortical granule {ECO:0000256|ARBA:ARBA00037865}. Sarcoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004564}. Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Cell surface {ECO:0000256|ARBA:ARBA00004241}. Cytoplasm, cytosol {ECO:0000256|ARBA:ARBA00004514}. Cytoplasmic vesicle, secretory vesicle, Cortical granule {ECO:0000256|ARBA:ARBA00037865}. Endoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004319, ECO:0000256|PIRNR:PIRNR002356}. Sarcoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004564}. Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}."
168,NOP56,"SUBCELLULAR LOCATION: Nucleus, nucleolus {ECO:0000269|PubMed:12429849, ECO:0000269|PubMed:12777385, ECO:0000269|PubMed:34516797, ECO:0000269|PubMed:9372940}. Cytoplasm {ECO:0000250|UniProtKB:Q9D6Z1}. Nucleus, nucleoplasm {ECO:0000305|PubMed:15574333}.,SUBCELLULAR LOCATION: Nucleus, nucleolus {ECO:0000256|ARBA:ARBA00004604}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}.,SUBCELLULAR LOCATION: Nucleus, nucleolus {ECO:0000256|ARBA:ARBA00004604}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}.,SUBCELLULAR LOCATION: Nucleus, nucleolus {ECO:0000256|ARBA:ARBA00004604}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}."
169,GNPTAB,"SUBCELLULAR LOCATION: [N-acetylglucosamine-1-phosphotransferase subunit alpha]: Golgi apparatus membrane {ECO:0000269|PubMed:16120602, ECO:0000269|PubMed:16200072, ECO:0000269|PubMed:21719679, ECO:0000269|PubMed:23733939, ECO:0000269|PubMed:24375680, ECO:0000269|PubMed:25788519, ECO:0000269|PubMed:28918368}; Single-pass type I membrane protein {ECO:0000305}.; SUBCELLULAR LOCATION: [N-acetylglucosamine-1-phosphotransferase subunit beta]: Golgi apparatus membrane {ECO:0000269|PubMed:16120602, ECO:0000269|PubMed:16200072, ECO:0000269|PubMed:21719679, ECO:0000269|PubMed:23733939, ECO:0000269|PubMed:24375680, ECO:0000269|PubMed:28918368}; Single-pass type II membrane protein {ECO:0000305}."
170,SERPINH1,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen.
171,CASP7,"SUBCELLULAR LOCATION: Cytoplasm, cytosol {ECO:0000269|PubMed:12824163, ECO:0000305|PubMed:21555521, ECO:0000305|PubMed:8576161}. Nucleus {ECO:0000269|PubMed:19617626, ECO:0000269|PubMed:21555521}. Secreted, extracellular space {ECO:0000250|UniProtKB:P97864}. Note=Following cleavage and activation by CASP1 or granzyme B (GZMB), secreted into the extracellular milieu by passing through the gasdermin-D (GSDMD) pores or perforin (PRF1) pore, respectively. {ECO:0000250|UniProtKB:P97864}."
172,THSD4,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:32855533}."
173,PLAT,"SUBCELLULAR LOCATION: Secreted, extracellular space.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|PIRNR:PIRNR001145}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|PIRNR:PIRNR001145}."
174,VCAN,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000305|PubMed:2583089}. Cell projection, cilium, photoreceptor outer segment {ECO:0000269|PubMed:29777959}. Secreted, extracellular space, extracellular matrix, interphotoreceptor matrix {ECO:0000269|PubMed:29777959}. Secreted {ECO:0000269|PubMed:25326458}.,SUBCELLULAR LOCATION: Cell projection, cilium, photoreceptor outer segment {ECO:0000256|ARBA:ARBA00004504}. Secreted, extracellular space, extracellular matrix, interphotoreceptor matrix {ECO:0000256|ARBA:ARBA00004593}."
175,EDEM3,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000255|PROSITE-ProRule:PRU10138}.
176,CGN,"SUBCELLULAR LOCATION: Cell junction, tight junction {ECO:0000250|UniProtKB:P59242}. Note=Localizes to the apical junction complex composed of tight and adherens junctions (By similarity). Colocalizes with SPEF1 at sites of cell-cell contact in intestinal epithelial cells (PubMed:12023291). {ECO:0000250|UniProtKB:P59242, ECO:0000269|PubMed:12023291}."
177,RPL17,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:23636399}.
178,PSAP,"SUBCELLULAR LOCATION: Lysosome {ECO:0000269|PubMed:14657016, ECO:0000269|PubMed:21835174, ECO:0000269|PubMed:22431521}.; SUBCELLULAR LOCATION: [Prosaposin]: Secreted {ECO:0000250|UniProtKB:Q61207}. Note=Secreted as a fully glycosylated 70 kDa protein composed of complex glycans. {ECO:0000250|UniProtKB:Q61207}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}."
179,MAN2B2,SUBCELLULAR LOCATION: Secreted {ECO:0000305}.
180,LMF2,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000250}; Multi-pass membrane protein {ECO:0000250}.
181,TLL2,SUBCELLULAR LOCATION: Secreted {ECO:0000305}.
182,TLL1,SUBCELLULAR LOCATION: Secreted {ECO:0000305}.
183,GNS,"SUBCELLULAR LOCATION: Lysosome.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|PIRNR:PIRNR036666}."
184,CTSD,"SUBCELLULAR LOCATION: Lysosome. Melanosome. Secreted, extracellular space. Note=Identified by mass spectrometry in melanosome fractions from stage I to stage IV. In aortic samples, detected as an extracellular protein loosely bound to the matrix (PubMed:20551380). {ECO:0000269|PubMed:20551380}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}."
185,ERAP1,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000305|PubMed:15908954}; Single-pass type II membrane protein {ECO:0000305|PubMed:15908954}.
186,TYR,"SUBCELLULAR LOCATION: Melanosome membrane {ECO:0000269|PubMed:12643545, ECO:0000269|PubMed:17081065}; Single-pass type I membrane protein {ECO:0000269|PubMed:12643545, ECO:0000269|PubMed:17081065}. Melanosome {ECO:0000250|UniProtKB:P11344}. Note=Proper trafficking to melanosome is regulated by SGSM2, ANKRD27, RAB9A, RAB32 and RAB38. {ECO:0000250|UniProtKB:P11344}.,SUBCELLULAR LOCATION: Melanosome membrane {ECO:0000256|ARBA:ARBA00004573}; Single-pass type I membrane protein {ECO:0000256|ARBA:ARBA00004573}.,SUBCELLULAR LOCATION: Melanosome membrane {ECO:0000256|ARBA:ARBA00004573}; Single-pass type I membrane protein {ECO:0000256|ARBA:ARBA00004573}.,SUBCELLULAR LOCATION: Melanosome membrane {ECO:0000256|ARBA:ARBA00004573}; Single-pass type I membrane protein {ECO:0000256|ARBA:ARBA00004573}.,SUBCELLULAR LOCATION: Melanosome membrane {ECO:0000256|ARBA:ARBA00004573}; Single-pass type I membrane protein {ECO:0000256|ARBA:ARBA00004573}.,SUBCELLULAR LOCATION: Melanosome membrane {ECO:0000256|ARBA:ARBA00004573}; Single-pass type I membrane protein {ECO:0000256|ARBA:ARBA00004573}.,SUBCELLULAR LOCATION: Melanosome membrane {ECO:0000256|ARBA:ARBA00004573}; Single-pass type I membrane protein {ECO:0000256|ARBA:ARBA00004573}.,SUBCELLULAR LOCATION: Melanosome membrane {ECO:0000256|ARBA:ARBA00004573}; Single-pass type I membrane protein {ECO:0000256|ARBA:ARBA00004573}."
187,HYOU1,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen.
188,PTPRQ,"SUBCELLULAR LOCATION: Cell projection, stereocilium {ECO:0000250|UniProtKB:P0C5E4}. Apical cell membrane; Single-pass type I membrane protein {ECO:0000255}. Basal cell membrane {ECO:0000269|PubMed:12837292}; Single-pass type I membrane protein {ECO:0000255}. Note=Detected at the stereocilium base and at the apical cell membrane in mature hair cells (By similarity). Forms ring-like structures in the stereocilium taper region (By similarity). Detected at the basal cell membrane in fetal kidney podocytes (PubMed:12837292). {ECO:0000250|UniProtKB:P0C5E4, ECO:0000269|PubMed:12837292}."
189,RPL3,"SUBCELLULAR LOCATION: Nucleus, nucleolus {ECO:0000269|PubMed:16963496}. Cytoplasm {ECO:0000269|PubMed:16963496, ECO:0000269|PubMed:23636399}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}.,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000256|ARBA:ARBA00004496}."
190,HEXB,"SUBCELLULAR LOCATION: Lysosome {ECO:0000269|PubMed:9694901}. Cytoplasmic vesicle, secretory vesicle, Cortical granule {ECO:0000250|UniProtKB:P20060}.,SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle, Cortical granule {ECO:0000256|ARBA:ARBA00037865}.,SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle, Cortical granule {ECO:0000256|ARBA:ARBA00037865}. Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle, Cortical granule {ECO:0000256|ARBA:ARBA00037865}.,SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle, Cortical granule {ECO:0000256|ARBA:ARBA00037865}."
191,F2,"SUBCELLULAR LOCATION: Secreted, extracellular space.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
192,ARSF,SUBCELLULAR LOCATION: Secreted {ECO:0000305}.
193,GNPTG,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:15532026}. Golgi apparatus {ECO:0000269|PubMed:15532026, ECO:0000269|PubMed:26108976, ECO:0000269|PubMed:27038293}."
194,TF,"SUBCELLULAR LOCATION: Secreted.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
195,COL12A1,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000250}."
196,SLCO3A1,"SUBCELLULAR LOCATION: [Isoform 1]: Basolateral cell membrane {ECO:0000269|PubMed:16971491}; Multi-pass membrane protein {ECO:0000255}. Note=Localized to the basolateral membrane of choroid plexus epithelium. {ECO:0000269|PubMed:16971491}.; SUBCELLULAR LOCATION: [Isoform 2]: Apical cell membrane {ECO:0000269|PubMed:16971491}; Multi-pass membrane protein {ECO:0000255}. Basal cell membrane {ECO:0000269|PubMed:35307651}; Multi-pass membrane protein {ECO:0000255}. Note=Localized to the basal membrane of Sertoli cells (PubMed:35307651). Localized to the apical membrane of choroid plexus epithelium (PubMed:16971491). {ECO:0000269|PubMed:16971491, ECO:0000269|PubMed:35307651}."
197,STT3B,SUBCELLULAR LOCATION: Endoplasmic reticulum {ECO:0000269|PubMed:12887896}. Endoplasmic reticulum membrane; Multi-pass membrane protein {ECO:0000250|UniProtKB:P39007}.
198,OLFML2B,SUBCELLULAR LOCATION: Secreted {ECO:0000250}.
199,COL1A1,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000255|PROSITE-ProRule:PRU00793}."
200,EXTL2,"SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000250}; Single-pass type II membrane protein {ECO:0000250}.; SUBCELLULAR LOCATION: [Processed exostosin-like 2]: Secreted. Note=A soluble form is found in the serum.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004648}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004648}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004648}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004648}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004648}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004648}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004648}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004648}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004648}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004648}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004648}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004648}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004648}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004648}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004648}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004648}."
201,GZMH,SUBCELLULAR LOCATION: Cytolytic granule {ECO:0000269|PubMed:23269243}.
202,FKBP14,"SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000255|PROSITE-ProRule:PRU10138, ECO:0000269|PubMed:22265013}."
203,PCYOX1,"SUBCELLULAR LOCATION: Lysosome {ECO:0000269|PubMed:10585463}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}."
204,M6PR,SUBCELLULAR LOCATION: Lysosome membrane; Single-pass type I membrane protein.
205,SEMA3C,SUBCELLULAR LOCATION: Secreted.
206,TMX4,SUBCELLULAR LOCATION: Nucleus inner membrane {ECO:0000250|UniProtKB:Q8C0L0}; Single-pass type I membrane protein {ECO:0000255}. Endoplasmic reticulum membrane {ECO:0000269|PubMed:22045338}; Single-pass type I membrane protein {ECO:0000255}.
207,ARSB,SUBCELLULAR LOCATION: Lysosome {ECO:0000250|UniProtKB:P50429}. Cell surface {ECO:0000250|UniProtKB:P50429}.
208,COL6A6,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000250}. Note=Deposed in the extracellular matrix of skeletal muscle. {ECO:0000250}."
209,LMAN2,SUBCELLULAR LOCATION: Endoplasmic reticulum-Golgi intermediate compartment membrane {ECO:0000269|PubMed:10444376}; Single-pass type I membrane protein {ECO:0000269|PubMed:10444376}. Golgi apparatus membrane {ECO:0000269|PubMed:10444376}; Single-pass membrane protein {ECO:0000269|PubMed:10444376}. Endoplasmic reticulum membrane {ECO:0000269|PubMed:10444376}; Single-pass type I membrane protein {ECO:0000269|PubMed:10444376}.
210,PPIAL4A,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000250|UniProtKB:P62937}.
211,LRPAP1,"SUBCELLULAR LOCATION: Rough endoplasmic reticulum lumen {ECO:0000269|PubMed:7774585}. Endoplasmic reticulum-Golgi intermediate compartment lumen {ECO:0000269|PubMed:7774585}. Golgi apparatus, cis-Golgi network {ECO:0000269|PubMed:7774585}. Golgi apparatus lumen {ECO:0000269|PubMed:7774585}. Endosome lumen {ECO:0000269|PubMed:7774585}. Cell surface {ECO:0000269|PubMed:11384978}. Note=May be associated with receptors at the cell surface. {ECO:0000269|PubMed:11384978}."
212,DNASE1L1,SUBCELLULAR LOCATION: Endoplasmic reticulum {ECO:0000269|PubMed:16107205}.
213,HSPD1,SUBCELLULAR LOCATION: Mitochondrion matrix.
214,SPANXB1,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:10906052}. Nucleus {ECO:0000269|PubMed:10906052}. Note=Associated with nuclear craters.
215,INTS12,SUBCELLULAR LOCATION: Nucleus {ECO:0000269|PubMed:23904267}.
216,APLF,"SUBCELLULAR LOCATION: Nucleus {ECO:0000269|PubMed:17353262, ECO:0000269|PubMed:17396150, ECO:0000269|PubMed:17507382, ECO:0000269|PubMed:23689425}. Chromosome {ECO:0000269|PubMed:18172500, ECO:0000269|PubMed:18474613, ECO:0000269|PubMed:21211721, ECO:0000269|PubMed:21211722, ECO:0000269|PubMed:23689425, ECO:0000269|PubMed:27063109}. Cytoplasm, cytosol {ECO:0000269|PubMed:17353262}. Note=Localizes to DNA damage sites (PubMed:18172500, PubMed:18474613, PubMed:21211721, PubMed:21211722, PubMed:23689425). Accumulates at single-strand breaks and double-strand breaks via the PBZ-type zinc fingers (PubMed:18172500). {ECO:0000269|PubMed:18172500, ECO:0000269|PubMed:18474613, ECO:0000269|PubMed:21211721, ECO:0000269|PubMed:21211722, ECO:0000269|PubMed:23689425}."
217,TUBA4A,"SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}."
218,THBS1,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:101549, ECO:0000269|PubMed:14568985, ECO:0000269|PubMed:6777381}. Cell surface {ECO:0000269|PubMed:6777381}. Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:18285447, ECO:0000269|PubMed:6341993}. Endoplasmic reticulum {ECO:0000250|UniProtKB:P35441}. Sarcoplasmic reticulum {ECO:0000250|UniProtKB:P35441}. Note=Secreted by thrombin-activated platelets and binds to the cell surface in the presence of extracellular Ca(2+) (PubMed:101549, PubMed:6777381). Incorporated into the extracellular matrix (ECM) of fibroblasts (PubMed:6341993). The C-terminal region in trimeric form is required for retention in the ECM (PubMed:18285447). Also detected in the endoplasmic reticulum and sarcoplasmic reticulum where it plays a role in the ER stress response (By similarity). {ECO:0000250|UniProtKB:P35441, ECO:0000269|PubMed:6341993, ECO:0000269|PubMed:6777381}."
219,TNFRSF11B,"SUBCELLULAR LOCATION: Secreted.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
220,B4GALT3,"SUBCELLULAR LOCATION: Golgi apparatus, Golgi stack membrane; Single-pass type II membrane protein. Note=Trans cisternae of Golgi stack."
221,EFEMP1,"SUBCELLULAR LOCATION: Secreted, extracellular space {ECO:0000250}. Secreted, extracellular space, extracellular matrix {ECO:0000250}. Note=Localizes to the lamina propria underneath the olfactory epithelium. {ECO:0000250}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}."
222,CDSN,"SUBCELLULAR LOCATION: Secreted. Note=Found in corneodesmosomes, the intercellular structures that are involved in desquamation.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
223,COL5A2,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000255|PROSITE-ProRule:PRU00793}."
224,RPL32,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:23636399}.
225,CHST3,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000250}; Single-pass type II membrane protein {ECO:0000250}.
226,LTBP3,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:12154076}. Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:16157329}. Note=Secretion occurs after coexpression with TGFB1 and requires complexing with 'Cys-33' of the TGFB1 propeptide. {ECO:0000269|PubMed:12154076}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}."
227,WDR13,SUBCELLULAR LOCATION: Nucleus {ECO:0000269|PubMed:12659815}.
228,LAMA1,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix, basement membrane. Note=Major component."
229,HNRNPK,"SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:1729596}. Nucleus, nucleoplasm {ECO:0000269|PubMed:16360036, ECO:0000269|PubMed:1729596, ECO:0000269|PubMed:18775702, ECO:0000269|PubMed:22721921}. Cell projection, podosome {ECO:0000269|PubMed:22721921}. Note=Recruited to p53/TP53-responsive promoters, in the presence of functional p53/TP53 (PubMed:16360036). In case of ASFV infection, there is a shift in the localization which becomes predominantly nuclear (PubMed:18775702).,SUBCELLULAR LOCATION: Cell projection, podosome {ECO:0000256|ARBA:ARBA00004188}. Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}.,SUBCELLULAR LOCATION: Cell projection, podosome {ECO:0000256|ARBA:ARBA00004188}. Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}.,SUBCELLULAR LOCATION: Cell projection, podosome {ECO:0000256|ARBA:ARBA00004188}. Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}.,SUBCELLULAR LOCATION: Cell projection, podosome {ECO:0000256|ARBA:ARBA00004188}. Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}.,SUBCELLULAR LOCATION: Cell projection, podosome {ECO:0000256|ARBA:ARBA00004188}. Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}.,SUBCELLULAR LOCATION: Cell projection, podosome {ECO:0000256|ARBA:ARBA00004188}. Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}.,SUBCELLULAR LOCATION: Cell projection, podosome {ECO:0000256|ARBA:ARBA00004188}. Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}.,SUBCELLULAR LOCATION: Cell projection, podosome {ECO:0000256|ARBA:ARBA00004188}. Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}.,SUBCELLULAR LOCATION: Cell projection, podosome {ECO:0000256|ARBA:ARBA00004188}. Cytoplasm {ECO:0000256|ARBA:ARBA00004496}. Nucleus, nucleoplasm {ECO:0000256|ARBA:ARBA00004642}."
230,FKBP7,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000255|PROSITE-ProRule:PRU10138}.
231,MPO,SUBCELLULAR LOCATION: Lysosome.
232,MMP13,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000305|PubMed:8576151}. Secreted {ECO:0000269|PubMed:8576151}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
233,ERGIC2,"SUBCELLULAR LOCATION: Endoplasmic reticulum-Golgi intermediate compartment membrane; Multi-pass membrane protein. Golgi apparatus, cis-Golgi network membrane; Multi-pass membrane protein. Endoplasmic reticulum membrane; Multi-pass membrane protein. Cytoplasm. Nucleus. Note=Cycles between the endoplasmic reticulum and the Golgi. According to a report, localizes to the nucleus (PubMed:11445006). Another report shows a partial localization in the nucleus (PubMed:17980171). {ECO:0000269|PubMed:11445006, ECO:0000269|PubMed:17980171}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}. Endoplasmic reticulum-Golgi intermediate compartment membrane {ECO:0000256|ARBA:ARBA00004457, ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|ARBA:ARBA00004457, ECO:0000256|RuleBase:RU369013}. Golgi apparatus membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}. Endoplasmic reticulum-Golgi intermediate compartment membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}. Golgi apparatus membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}. Endoplasmic reticulum-Golgi intermediate compartment membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}. Golgi apparatus membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}. Endoplasmic reticulum-Golgi intermediate compartment membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}. Golgi apparatus membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}. Endoplasmic reticulum-Golgi intermediate compartment membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}. Golgi apparatus membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}. Endoplasmic reticulum-Golgi intermediate compartment membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}. Golgi apparatus membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}. Endoplasmic reticulum-Golgi intermediate compartment membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}. Golgi apparatus membrane {ECO:0000256|RuleBase:RU369013}; Multi-pass membrane protein {ECO:0000256|RuleBase:RU369013}."
234,ANGPTL1,SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:10025962}.
235,ACP2,"SUBCELLULAR LOCATION: Lysosome membrane {ECO:0000269|PubMed:2684640, ECO:0000269|PubMed:3056714}; Single-pass membrane protein {ECO:0000255}; Lumenal side {ECO:0000305|PubMed:2684640, ECO:0000305|PubMed:3056714}. Lysosome lumen. Note=The soluble form arises by proteolytic processing of the membrane-bound form. {ECO:0000269|PubMed:2684640, ECO:0000269|PubMed:3056714}.,SUBCELLULAR LOCATION: Lysosome lumen {ECO:0000256|ARBA:ARBA00004227}. Lysosome membrane {ECO:0000256|ARBA:ARBA00037852}; Single-pass membrane protein {ECO:0000256|ARBA:ARBA00037852}; Lumenal side {ECO:0000256|ARBA:ARBA00037852}.,SUBCELLULAR LOCATION: Lysosome lumen {ECO:0000256|ARBA:ARBA00004227}. Lysosome membrane {ECO:0000256|ARBA:ARBA00037852}; Single-pass membrane protein {ECO:0000256|ARBA:ARBA00037852}; Lumenal side {ECO:0000256|ARBA:ARBA00037852}.,SUBCELLULAR LOCATION: Lysosome lumen {ECO:0000256|ARBA:ARBA00004227}. Lysosome membrane {ECO:0000256|ARBA:ARBA00037852}; Single-pass membrane protein {ECO:0000256|ARBA:ARBA00037852}; Lumenal side {ECO:0000256|ARBA:ARBA00037852}.,SUBCELLULAR LOCATION: Lysosome lumen {ECO:0000256|ARBA:ARBA00004227}. Lysosome membrane {ECO:0000256|ARBA:ARBA00037852}; Single-pass membrane protein {ECO:0000256|ARBA:ARBA00037852}; Lumenal side {ECO:0000256|ARBA:ARBA00037852}.,SUBCELLULAR LOCATION: Lysosome lumen {ECO:0000256|ARBA:ARBA00004227}. Lysosome membrane {ECO:0000256|ARBA:ARBA00037852}; Single-pass membrane protein {ECO:0000256|ARBA:ARBA00037852}; Lumenal side {ECO:0000256|ARBA:ARBA00037852}.,SUBCELLULAR LOCATION: Lysosome lumen {ECO:0000256|ARBA:ARBA00004227}. Lysosome membrane {ECO:0000256|ARBA:ARBA00037852}; Single-pass membrane protein {ECO:0000256|ARBA:ARBA00037852}; Lumenal side {ECO:0000256|ARBA:ARBA00037852}.,SUBCELLULAR LOCATION: Lysosome lumen {ECO:0000256|ARBA:ARBA00004227}. Lysosome membrane {ECO:0000256|ARBA:ARBA00037852}; Single-pass membrane protein {ECO:0000256|ARBA:ARBA00037852}; Lumenal side {ECO:0000256|ARBA:ARBA00037852}.,SUBCELLULAR LOCATION: Lysosome lumen {ECO:0000256|ARBA:ARBA00004227}. Lysosome membrane {ECO:0000256|ARBA:ARBA00037852}; Single-pass membrane protein {ECO:0000256|ARBA:ARBA00037852}; Lumenal side {ECO:0000256|ARBA:ARBA00037852}."
236,SLC29A2,SUBCELLULAR LOCATION: Apical cell membrane {ECO:0000269|PubMed:23639800}; Multi-pass membrane protein {ECO:0000305}. Basolateral cell membrane {ECO:0000269|PubMed:12527552}; Multi-pass membrane protein {ECO:0000305}. Note=Localized to the apical membrane of Sertoli cells. {ECO:0000269|PubMed:23639800}.
237,GLCE,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000250|UniProtKB:Q9EPS3}; Single-pass type II membrane protein {ECO:0000250|UniProtKB:Q9EPS3}.
238,CTSA,SUBCELLULAR LOCATION: Lysosome.
239,ADAMTSL4,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:21989719}. Note=Colocalizes with FMN1 microfibrils in the eye ECM."
240,OLFML3,SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:15280020}.
241,UST,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000250}; Single-pass type II membrane protein {ECO:0000250}.
242,TRAP1,"SUBCELLULAR LOCATION: Mitochondrion {ECO:0000269|PubMed:23564345}. Mitochondrion inner membrane {ECO:0000269|PubMed:23564345}. Mitochondrion matrix {ECO:0000269|PubMed:23564345}.,SUBCELLULAR LOCATION: Mitochondrion {ECO:0000256|ARBA:ARBA00004173}.,SUBCELLULAR LOCATION: Mitochondrion {ECO:0000256|ARBA:ARBA00004173}.,SUBCELLULAR LOCATION: Mitochondrion {ECO:0000256|ARBA:ARBA00004173}.,SUBCELLULAR LOCATION: Mitochondrion {ECO:0000256|ARBA:ARBA00004173}.,SUBCELLULAR LOCATION: Mitochondrion {ECO:0000256|ARBA:ARBA00004173}."
243,CERCAM,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000255|PROSITE-ProRule:PRU10138}.
244,LGALS3BP,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:8390986}. Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:9501082}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}."
245,SERPING1,SUBCELLULAR LOCATION: Secreted.
246,NPTX2,SUBCELLULAR LOCATION: Secreted {ECO:0000250}.
247,FGA,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:19296670, ECO:0000269|PubMed:9628725}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
248,CTSL1,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.
249,INHBE,SUBCELLULAR LOCATION: Secreted {ECO:0000250}.
250,IGFBP3,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:20353938}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
251,UGGT1,"SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000255|PROSITE-ProRule:PRU10138, ECO:0000269|PubMed:10694380}. Endoplasmic reticulum-Golgi intermediate compartment {ECO:0000255|PROSITE-ProRule:PRU10138, ECO:0000269|PubMed:10694380}."
252,PBXIP1,"SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000269|PubMed:12360403, ECO:0000269|PubMed:17043237}. Nucleus {ECO:0000269|PubMed:10825160, ECO:0000269|PubMed:12360403}. Note=Shuttles between the nucleus and the cytosol (PubMed:12360403). Mainly localized in the cytoplasm, associated with microtubules (PubMed:10825160, PubMed:12360403). Detected in small amounts in the nucleus (PubMed:10825160)."
253,B4GALT5,"SUBCELLULAR LOCATION: Golgi apparatus, Golgi stack membrane {ECO:0000250|UniProtKB:P15291}; Single-pass type II membrane protein. Golgi apparatus {ECO:0000250|UniProtKB:A0A1S6M251}. Note=Trans cisternae of Golgi stack. {ECO:0000250|UniProtKB:P15291}."
254,SEL1L,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000269|PubMed:16186509}; Single-pass type I membrane protein {ECO:0000269|PubMed:16186509}.
255,GLA,"SUBCELLULAR LOCATION: Lysosome.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}.,SUBCELLULAR LOCATION: Lysosome {ECO:0000256|ARBA:ARBA00004371}."
256,HSD17B2,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000269|PubMed:10385431}; Single-pass type II membrane protein {ECO:0000305}.
257,MAN2A1,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000250|UniProtKB:P28494}; Single-pass type II membrane protein {ECO:0000250|UniProtKB:P28494}.
258,ANGPTL2,SUBCELLULAR LOCATION: Secreted.
259,SULF1,"SUBCELLULAR LOCATION: Endoplasmic reticulum {ECO:0000250|UniProtKB:Q8VI60}. Golgi apparatus, Golgi stack {ECO:0000250|UniProtKB:Q8VI60}. Cell surface {ECO:0000269|PubMed:12368295, ECO:0000269|PubMed:12686563}.; SUBCELLULAR LOCATION: [Extracellular sulfatase Sulf-2 secreted form]: Secreted {ECO:0000269|PubMed:12368295}.,SUBCELLULAR LOCATION: Cell surface {ECO:0000256|ARBA:ARBA00004241}. Endoplasmic reticulum {ECO:0000256|ARBA:ARBA00004240}. Golgi apparatus, Golgi stack {ECO:0000256|ARBA:ARBA00004348}.,SUBCELLULAR LOCATION: Cell surface {ECO:0000256|ARBA:ARBA00004241}. Endoplasmic reticulum {ECO:0000256|ARBA:ARBA00004240}. Golgi apparatus, Golgi stack {ECO:0000256|ARBA:ARBA00004348}."
260,FLNA,"SUBCELLULAR LOCATION: Cytoplasm, cell cortex {ECO:0000269|PubMed:16291724}. Cytoplasm, cytoskeleton {ECO:0000250|UniProtKB:Q8BTM8}. Perikaryon {ECO:0000250|UniProtKB:Q8BTM8}. Cell projection, growth cone {ECO:0000250|UniProtKB:Q8BTM8}. Note=Colocalizes with CPMR1 in the central region of DRG neuron growth cone (By similarity). Following SEMA3A stimulation of DRG neurons, colocalizes with F-actin (By similarity). {ECO:0000250|UniProtKB:Q8BTM8}."
261,LAMC1,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix, basement membrane."
262,CES1,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000269|PubMed:10562416}. Cytoplasm {ECO:0000269|PubMed:16024911}. Lipid droplet {ECO:0000269|PubMed:16024911}. Note=Moves from cytoplasm to lipid droplets upon lipid loading. Associates with lipid droplets independently of triglycerides (TG) content of the droplets and hydrolyzes cholesteryl esters more efficiently from mixed droplets. {ECO:0000269|PubMed:16024911}.
263,BIRC6,"SUBCELLULAR LOCATION: Golgi apparatus, trans-Golgi network membrane {ECO:0000269|PubMed:18329369}. Endosome {ECO:0000269|PubMed:18329369}. Cytoplasm, cytoskeleton, spindle pole {ECO:0000269|PubMed:18329369}. Cytoplasm, cytoskeleton, microtubule organizing center, centrosome {ECO:0000269|PubMed:18329369}. Midbody, Midbody ring {ECO:0000269|PubMed:18329369}. Note=Exhibits cell cycle-dependent localization. Concentrates in a pericentriolar compartment in interphase, moves partially to spindle poles in metaphase, and finally localizes to the spindle midzone and the midbody in telophase and during cytokinesis. On the midbody, localizes to the midbody ring, also called Flemming body (PubMed:18329369). In interphase cells, localizes to the trans-Golgi network membrane and endosomes. During cytokinesis, a fraction moves to the midzone where it specifically arrives at the midbody ring. After abscission completion, travels with the midbody remnant into one daughter cell, and remains bound to it until a new midbody ring is formed during the next cell division (PubMed:18329369). {ECO:0000269|PubMed:18329369}."
264,SUMF1,"SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000269|PubMed:15657036, ECO:0000269|PubMed:15962010, ECO:0000269|PubMed:18266766, ECO:0000269|PubMed:21224894}.,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004319}.,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004319}."
265,ACTB,"SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000269|PubMed:11687588, ECO:0000269|PubMed:17289661}. Nucleus {ECO:0000269|PubMed:11687588, ECO:0000269|PubMed:29925947}. Note=Localized in cytoplasmic mRNP granules containing untranslated mRNAs. {ECO:0000269|PubMed:17289661}."
266,FLG2,"SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:19384417}. Cytoplasmic granule {ECO:0000269|PubMed:19384417}. Note=In the stratum corneum of the epidermis, dispersed diffusely throughout the cytoplasm, while in the stratum granulosum, localized within keratohyalin granules (PubMed:19384417, PubMed:21531719). In granular keratinocytes and in lower corneocytes, colocalizes with calpain-1/CAPN1. {ECO:0000269|PubMed:19384417, ECO:0000269|PubMed:21531719}."
267,SLIT2,SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:10102268}. Note=The C-terminal cleavage protein is more diffusible than the larger N-terminal protein that is more tightly cell associated.
268,SLC22A4,"SUBCELLULAR LOCATION: Apical cell membrane {ECO:0000269|PubMed:20601551}; Multi-pass membrane protein {ECO:0000305}. Basal cell membrane {ECO:0000269|PubMed:35307651}; Multi-pass membrane protein {ECO:0000305}. Mitochondrion membrane {ECO:0000269|PubMed:16729965}; Multi-pass membrane protein {ECO:0000305}. Note=Localized to the apical membrane of small intestines (PubMed:20601551). Localized to the basal membrane of Sertoli cells (PubMed:35307651). {ECO:0000269|PubMed:20601551, ECO:0000269|PubMed:35307651}."
269,RPL37A,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:23636399}.
270,SLIT3,SUBCELLULAR LOCATION: Secreted {ECO:0000250}.
271,RPS23,"SUBCELLULAR LOCATION: Cytoplasm, cytosol {ECO:0000269|PubMed:25957688}. Cytoplasm {ECO:0000305|PubMed:23636399, ECO:0000305|PubMed:25901680}. Rough endoplasmic reticulum {ECO:0000250|UniProtKB:Q6SA96}. Nucleus, nucleolus {ECO:0000269|PubMed:34516797}. Note=Detected on cytosolic polysomes (PubMed:25957688). Detected in ribosomes that are associated with the rough endoplasmic reticulum (By similarity). {ECO:0000250|UniProtKB:Q6SA96, ECO:0000269|PubMed:25957688}.,SUBCELLULAR LOCATION: Rough endoplasmic reticulum {ECO:0000256|ARBA:ARBA00004427}.,SUBCELLULAR LOCATION: Rough endoplasmic reticulum {ECO:0000256|ARBA:ARBA00004427}."
272,LOXL4,"SUBCELLULAR LOCATION: Secreted, extracellular space {ECO:0000305}."
273,PIGT,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000269|PubMed:11483512}; Single-pass type I membrane protein {ECO:0000269|PubMed:11483512}.
274,GPI,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:11437381}. Secreted {ECO:0000269|PubMed:11437381}.
275,ENTPD7,SUBCELLULAR LOCATION: Cytoplasmic vesicle membrane {ECO:0000269|PubMed:11278936}; Multi-pass membrane protein {ECO:0000255}.
276,SERINC1,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000250|UniProtKB:Q7TNK0}; Multi-pass membrane protein {ECO:0000250|UniProtKB:Q7TNK0}.
277,LUM,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000250}."
278,OSTM1,SUBCELLULAR LOCATION: Lysosome membrane {ECO:0000269|PubMed:21527911}; Single-pass type I membrane protein {ECO:0000269|PubMed:21527911}. Note=Requires CLCN7 to travel to lysosomes.
279,OLFML2A,SUBCELLULAR LOCATION: Secreted {ECO:0000250|UniProtKB:Q8BHP7}. Note=Localizes to the podocyte major processes. Colocalized with the major process protein VIM throughout podocyte development. {ECO:0000269|PubMed:22913984}.
280,LIPG,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:10318835}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
281,A2M,SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:6203908}.
282,COL2A1,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000255|PROSITE-ProRule:PRU00793}."
283,FBLN2,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}."
284,FUT11,"SUBCELLULAR LOCATION: Golgi apparatus, Golgi stack membrane {ECO:0000250}; Single-pass type II membrane protein {ECO:0000250}."
285,TPI1,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000255|PROSITE-ProRule:PRU10127}.
286,HPX,SUBCELLULAR LOCATION: Secreted.
287,C4A,"SUBCELLULAR LOCATION: Secreted. Synapse {ECO:0000269|PubMed:26814963}. Cell projection, axon {ECO:0000269|PubMed:26814963}. Cell projection, dendrite {ECO:0000269|PubMed:26814963}.,SUBCELLULAR LOCATION: Cell projection, axon {ECO:0000256|ARBA:ARBA00004489}. Cell projection, dendrite {ECO:0000256|ARBA:ARBA00004279}. Secreted {ECO:0000256|ARBA:ARBA00004613}. Synapse {ECO:0000256|ARBA:ARBA00034103}.,SUBCELLULAR LOCATION: Cell projection, axon {ECO:0000256|ARBA:ARBA00004489}. Cell projection, dendrite {ECO:0000256|ARBA:ARBA00004279}. Secreted {ECO:0000256|ARBA:ARBA00004613}. Synapse {ECO:0000256|ARBA:ARBA00034103}.,SUBCELLULAR LOCATION: Cell projection, axon {ECO:0000256|ARBA:ARBA00004489}. Cell projection, dendrite {ECO:0000256|ARBA:ARBA00004279}. Secreted {ECO:0000256|ARBA:ARBA00004613}. Synapse {ECO:0000256|ARBA:ARBA00034103}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
288,SEMA3A,SUBCELLULAR LOCATION: Secreted {ECO:0000250}.
289,SFRS3,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.
290,CCT4,"SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:17081065, ECO:0000269|PubMed:20080638}. Melanosome {ECO:0000269|PubMed:17081065}. Cytoplasm, cytoskeleton, microtubule organizing center, centrosome {ECO:0000269|PubMed:14654843, ECO:0000269|PubMed:20080638}. Cytoplasm, cytoskeleton, cilium basal body {ECO:0000250|UniProtKB:P80315}. Note=Identified by mass spectrometry in melanosome fractions from stage I to stage IV. {ECO:0000269|PubMed:17081065}."
291,SERPINE2,"SUBCELLULAR LOCATION: Secreted, extracellular space."
292,CHST7,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000250}; Single-pass type II membrane protein {ECO:0000250}.
293,TXNDC15,"SUBCELLULAR LOCATION: Cell projection, cilium membrane {ECO:0000250|UniProtKB:Q6P6J9}; Single-pass type I membrane protein {ECO:0000255}."
294,RALY,SUBCELLULAR LOCATION: Nucleus {ECO:0000250|UniProtKB:Q64012}.
295,LAMP3,"SUBCELLULAR LOCATION: Cell surface {ECO:0000269|PubMed:27329040}. Lysosome membrane {ECO:0000269|PubMed:21930964, ECO:0000269|PubMed:9768752}; Single-pass type I membrane protein {ECO:0000255}. Cytoplasmic vesicle membrane {ECO:0000269|PubMed:9768752}; Single-pass type I membrane protein {ECO:0000255}. Early endosome membrane {ECO:0000269|PubMed:21810281}; Single-pass type I membrane protein {ECO:0000255}. Note=During dendritic cell maturation, detected on cytoplasmic vesicles (the MHC II compartment) that contain MHC II proteins, LAMP1, LAMP2 and LAMP3 (PubMed:9768752). Detected on lysosomes in mature dendritic cells (PubMed:9768752). {ECO:0000269|PubMed:9768752}.,SUBCELLULAR LOCATION: Lysosome membrane {ECO:0000256|PROSITE-ProRule:PRU00740}; Single-pass type I membrane protein {ECO:0000256|PROSITE-ProRule:PRU00740}.,SUBCELLULAR LOCATION: Endosome membrane {ECO:0000256|ARBA:ARBA00004530}; Single-pass type I membrane protein {ECO:0000256|ARBA:ARBA00004530}. Lysosome membrane {ECO:0000256|PROSITE-ProRule:PRU00740}; Single-pass type I membrane protein {ECO:0000256|PROSITE-ProRule:PRU00740}.,SUBCELLULAR LOCATION: Lysosome membrane {ECO:0000256|PROSITE-ProRule:PRU00740}; Single-pass type I membrane protein {ECO:0000256|PROSITE-ProRule:PRU00740}."
296,PI16,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:15344909, ECO:0000269|PubMed:17062675, ECO:0000269|PubMed:22171320}."
297,FICD,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000269|PubMed:25601083}; Single-pass type II membrane protein {ECO:0000305|PubMed:25601083}.
298,SLCO4C1,"SUBCELLULAR LOCATION: Basolateral cell membrane {ECO:0000269|PubMed:30865704}; Multi-pass membrane protein {ECO:0000250|UniProtKB:Q71MB6}. Note=Detected at the basolateral membrane of the proximal tubule cell in the kidney. {ECO:0000250|UniProtKB:Q71MB6, ECO:0000269|PubMed:30865704}."
299,UQCRC2,SUBCELLULAR LOCATION: Mitochondrion inner membrane {ECO:0000250|UniProtKB:P07257}; Peripheral membrane protein {ECO:0000250|UniProtKB:P07257}; Matrix side {ECO:0000250|UniProtKB:P07257}.
300,CREG1,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:10815803}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
301,EEF1D,SUBCELLULAR LOCATION: [Isoform 2]: Nucleus {ECO:0000269|PubMed:21597468}.
302,RPL27A,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:23636399}.
303,FSTL1,SUBCELLULAR LOCATION: Secreted {ECO:0000305}.
304,CRTAP,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000250}."
305,DPP7,SUBCELLULAR LOCATION: Lysosome {ECO:0000269|PubMed:15487984}. Cytoplasmic vesicle {ECO:0000269|PubMed:15487984}. Secreted {ECO:0000269|PubMed:15487984}.
306,NAV2,SUBCELLULAR LOCATION: Nucleus {ECO:0000269|PubMed:12214280}.
307,ZNF844,SUBCELLULAR LOCATION: Nucleus {ECO:0000250}.
308,VEGFC,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:9233800}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
309,RCN1,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen.
310,TFPI,"SUBCELLULAR LOCATION: [Isoform Alpha]: Secreted.; SUBCELLULAR LOCATION: [Isoform Beta]: Microsome membrane {ECO:0000269|PubMed:22144186}; Lipid-anchor, GPI-anchor {ECO:0000269|PubMed:22144186}."
311,DMTF1,"SUBCELLULAR LOCATION: Nucleus {ECO:0000255|PROSITE-ProRule:PRU00625, ECO:0000269|PubMed:17936562}."
312,ASPN,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:17827158}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498, ECO:0000256|PIRNR:PIRNR002490}."
313,P4HA2,"SUBCELLULAR LOCATION: Endoplasmic reticulum lumen.,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004319}."
314,MPHOSPH9,"SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton, microtubule organizing center, centrosome, centriole {ECO:0000269|PubMed:21399614, ECO:0000269|PubMed:30375385}. Golgi apparatus membrane {ECO:0000305}; Peripheral membrane protein {ECO:0000305}. Cytoplasm, cytoskeleton, microtubule organizing center, centrosome {ECO:0000269|PubMed:30375385}. Note=Localizes to the distal and proximal end of centriole pairs in duplicated centrosomes. In ciliated cells, localizes to the distal and proximal end of daughter centriole and proximal of the mother centriole but not in the distal end of the mother centriole (PubMed:21399614). Recruited by KIF24 to the distal end of mother centriole where it forms a ring-like structure (PubMed:30375385). {ECO:0000269|PubMed:21399614, ECO:0000269|PubMed:30375385}."
315,XRCC6,"SUBCELLULAR LOCATION: Nucleus {ECO:0000269|PubMed:22442688}. Chromosome {ECO:0000269|PubMed:22442688}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}."
316,NELL1,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:21723284}. Nucleus envelope {ECO:0000269|PubMed:21723284}. Secreted {ECO:0000250|UniProtKB:Q62919}. Note=Colocalizes with ATRAID on the nuclear envelope and the perinuclear region.
317,ASPH,"SUBCELLULAR LOCATION: [Isoform 1]: Endoplasmic reticulum membrane; Single-pass type II membrane protein {ECO:0000250|UniProtKB:Q28056}.; SUBCELLULAR LOCATION: [Isoform 4]: Sarcoplasmic reticulum membrane; Single-pass type II membrane protein {ECO:0000305|PubMed:15485681}.; SUBCELLULAR LOCATION: [Isoform 8]: Endoplasmic reticulum membrane; Single-pass type II membrane protein {ECO:0000269|PubMed:22586105}.,SUBCELLULAR LOCATION: Sarcoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004157}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004157}.,SUBCELLULAR LOCATION: Sarcoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004157}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004157}.,SUBCELLULAR LOCATION: Sarcoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004157}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004157}.,SUBCELLULAR LOCATION: Sarcoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004157}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004157}.,SUBCELLULAR LOCATION: Sarcoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004157}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004157}.,SUBCELLULAR LOCATION: Sarcoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004157}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004157}.,SUBCELLULAR LOCATION: Sarcoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004157}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004157}.,SUBCELLULAR LOCATION: Sarcoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004157}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004157}.,SUBCELLULAR LOCATION: Sarcoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004157}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004157}."
318,NTN3,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000250}."
319,TNC,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}."
320,RPS15A,"SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:23636399}. Nucleus, nucleolus {ECO:0000269|PubMed:34516797}."
321,AGPS,"SUBCELLULAR LOCATION: Peroxisome membrane {ECO:0000250|UniProtKB:P97275}. Peroxisome {ECO:0000250|UniProtKB:P97275}.,SUBCELLULAR LOCATION: Peroxisome {ECO:0000256|RuleBase:RU363113}.,SUBCELLULAR LOCATION: Peroxisome {ECO:0000256|RuleBase:RU363113}.,SUBCELLULAR LOCATION: Peroxisome {ECO:0000256|RuleBase:RU363113}.,SUBCELLULAR LOCATION: Peroxisome {ECO:0000256|RuleBase:RU363113}.,SUBCELLULAR LOCATION: Peroxisome {ECO:0000256|RuleBase:RU363113}.,SUBCELLULAR LOCATION: Peroxisome {ECO:0000256|RuleBase:RU363113}.,SUBCELLULAR LOCATION: Peroxisome {ECO:0000256|RuleBase:RU363113}.,SUBCELLULAR LOCATION: Peroxisome {ECO:0000256|RuleBase:RU363113}.,SUBCELLULAR LOCATION: Peroxisome {ECO:0000256|RuleBase:RU363113}.,SUBCELLULAR LOCATION: Peroxisome {ECO:0000256|RuleBase:RU363113}.,SUBCELLULAR LOCATION: Peroxisome {ECO:0000256|RuleBase:RU363113}.,SUBCELLULAR LOCATION: Peroxisome {ECO:0000256|RuleBase:RU363113}.,SUBCELLULAR LOCATION: Peroxisome {ECO:0000256|RuleBase:RU363113}.,SUBCELLULAR LOCATION: Peroxisome {ECO:0000256|RuleBase:RU363113}."
322,MYH9,"SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000269|PubMed:22693546}. Cytoplasm, cell cortex {ECO:0000250|UniProtKB:Q8VDD5}. Cytoplasmic vesicle, secretory vesicle, Cortical granule {ECO:0000250|UniProtKB:Q8VDD5}. Note=Colocalizes with actin filaments at lamellipodia margins and at the leading edge of migrating cells (PubMed:20052411). In retinal pigment epithelial cells, predominantly localized to stress fiber-like structures with some localization to cytoplasmic puncta (PubMed:27331610). {ECO:0000269|PubMed:20052411, ECO:0000269|PubMed:27331610}.,SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle, Cortical granule {ECO:0000256|ARBA:ARBA00037865}.,SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle, Cortical granule {ECO:0000256|ARBA:ARBA00037865}.,SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle, Cortical granule {ECO:0000256|ARBA:ARBA00037865}."
323,PRSS35,SUBCELLULAR LOCATION: Secreted {ECO:0000305}.
324,COL5A1,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000255|PROSITE-ProRule:PRU00793}."
325,ABCC2,"SUBCELLULAR LOCATION: Apical cell membrane {ECO:0000269|PubMed:10220572, ECO:0000269|PubMed:11093739, ECO:0000269|PubMed:28408210}; Multi-pass membrane protein {ECO:0000255}. Note=Localized to the apical membrane of enterocytes. {ECO:0000269|PubMed:28408210}."
326,SERPINA3,SUBCELLULAR LOCATION: Secreted.
327,VTN,"SUBCELLULAR LOCATION: Secreted, extracellular space {ECO:0000269|PubMed:2448300, ECO:0000269|PubMed:29567995}.; SUBCELLULAR LOCATION: Parasitophorous vacuole {ECO:0000269|PubMed:29567995}. Note=(Microbial infection) In P.falciparum-infected red blood cells, VTN internalization is detected at the early trophozoite stage (PubMed:29567995). Colocalizes with SERA5 at the schizont stage and with SERA5 P47 at the merozoite surface (PubMed:29567995). {ECO:0000269|PubMed:29567995}."
328,TUBB6,"SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000250}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}.,SUBCELLULAR LOCATION: Cytoplasm, cytoskeleton {ECO:0000256|ARBA:ARBA00004245}."
329,DBH,"SUBCELLULAR LOCATION: [Soluble dopamine beta-hydroxylase]: Cytoplasmic vesicle, secretory vesicle lumen {ECO:0000269|PubMed:7961964}. Cytoplasmic vesicle, secretory vesicle, chromaffin granule lumen {ECO:0000269|PubMed:7961964}. Secreted {ECO:0000269|PubMed:7961964, ECO:0000269|PubMed:8546710}.; SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle membrane {ECO:0000269|PubMed:7961964}; Single-pass type II membrane protein {ECO:0000305}. Cytoplasmic vesicle, secretory vesicle, chromaffin granule membrane {ECO:0000269|PubMed:7961964}; Single-pass type II membrane protein {ECO:0000305}.,SUBCELLULAR LOCATION: Cytoplasmic vesicle, secretory vesicle lumen {ECO:0000256|ARBA:ARBA00004263}. Cytoplasmic vesicle, secretory vesicle membrane {ECO:0000256|ARBA:ARBA00004213}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004213}. Cytoplasmic vesicle, secretory vesicle, chromaffin granule lumen {ECO:0000256|ARBA:ARBA00004553}. Cytoplasmic vesicle, secretory vesicle, chromaffin granule membrane {ECO:0000256|ARBA:ARBA00004351}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004351}."
330,SMPD1,"SUBCELLULAR LOCATION: Lysosome {ECO:0000269|PubMed:16787399, ECO:0000269|PubMed:18815062, ECO:0000269|PubMed:20530211, ECO:0000269|PubMed:20807762, ECO:0000269|PubMed:27659707, ECO:0000269|PubMed:9660788}. Lipid droplet {ECO:0000269|PubMed:25339683}. Secreted {ECO:0000269|PubMed:12563314, ECO:0000269|PubMed:16787399, ECO:0000269|PubMed:17303575, ECO:0000269|PubMed:20530211, ECO:0000269|PubMed:20807762, ECO:0000269|PubMed:22573858, ECO:0000269|PubMed:27659707, ECO:0000269|PubMed:8702487, ECO:0000269|PubMed:9030779, ECO:0000269|PubMed:9660788}. Note=The secreted form is induced in a time- and dose-dependent by IL1B and TNF as well as stress and viral infection. This increase of the secreted form seems to be due to exocytosis of the lysosomal form and is Ca(2+)-dependent (PubMed:20530211, PubMed:20807762, PubMed:22573858). Secretion is dependent of phosphorylation at Ser-510 (PubMed:17303575). Secretion is induced by inflammatory mediators such as IL1B, IFNG or TNF as well as infection with bacteria and viruses (PubMed:12563314, PubMed:20807762). {ECO:0000269|PubMed:12563314, ECO:0000269|PubMed:17303575, ECO:0000269|PubMed:20530211, ECO:0000269|PubMed:20807762, ECO:0000269|PubMed:22573858}.; SUBCELLULAR LOCATION: [Sphingomyelin phosphodiesterase, processed form]: Secreted, extracellular space {ECO:0000250|UniProtKB:Q04519}. Note=This form is generated following cleavage by CASP7. {ECO:0000250|UniProtKB:Q04519}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
331,CPZ,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:10671522}."
332,TMED9,"SUBCELLULAR LOCATION: Endoplasmic reticulum membrane; Single-pass type I membrane protein. Golgi apparatus, cis-Golgi network membrane; Single-pass type I membrane protein. Endoplasmic reticulum-Golgi intermediate compartment membrane; Single-pass type I membrane protein. Golgi apparatus, trans-Golgi network membrane {ECO:0000250}; Single-pass type I membrane protein {ECO:0000250}. Note=Cycles between compartments of the early secretatory pathway."
333,CP,SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:16150804}. Note=Colocalizes with GCP1 in secretory intracellular compartments. {ECO:0000250|UniProtKB:P13635}.
334,MXRA5,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:25326458, ECO:0000269|PubMed:27599751}."
335,AKAP11,"SUBCELLULAR LOCATION: Cytoplasm. Cytoplasm, cytoskeleton, microtubule organizing center, centrosome. Note=Cytoplasmic in premeiotic pachytene spermatocytes and in the centrosome of developing postmeiotic germ cells, while a midpiece/centrosome localization was found in elongating spermatocytes and mature sperm."
336,MMRN1,SUBCELLULAR LOCATION: Secreted {ECO:0000305}.
337,ERAP2,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000269|PubMed:12799365}; Single-pass type II membrane protein {ECO:0000269|PubMed:12799365}.
338,COL7A1,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix, basement membrane."
339,NAGLU,SUBCELLULAR LOCATION: Lysosome.
340,PLTP,SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:19321130}. Nucleus {ECO:0000269|PubMed:19321130}. Note=Nuclear export is XPO1/CRM1-dependent. {ECO:0000269|PubMed:19321130}.
341,MFAP4,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:26601954}."
342,MGAT4B,"SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000250|UniProtKB:Q9D4R2}; Single-pass type II membrane protein {ECO:0000250|UniProtKB:Q9D4R2}. Note=A processed soluble form also exists. {ECO:0000269|PubMed:17006639}.,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}.,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}.,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}.,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}.,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}.,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}.,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000256|ARBA:ARBA00004323}; Single-pass type II membrane protein {ECO:0000256|ARBA:ARBA00004323}."
343,PRKCSH,"SUBCELLULAR LOCATION: Endoplasmic reticulum {ECO:0000255|PROSITE-ProRule:PRU10138, ECO:0000305|PubMed:10929008}."
344,C1QTNF2,SUBCELLULAR LOCATION: Secreted {ECO:0000305}.
345,ZC3HAV1,"SUBCELLULAR LOCATION: [Isoform 1]: Cytoplasm {ECO:0000250|UniProtKB:Q8K3Y6}. Nucleus {ECO:0000250|UniProtKB:Q8K3Y6}. Note=Localizes in the cytoplasm at steady state, but shuttles between nucleus and cytoplasm in a XPO1-dependent manner. {ECO:0000250|UniProtKB:Q8K3Y6}.; SUBCELLULAR LOCATION: [Isoform 2]: Cytoplasm {ECO:0000269|PubMed:21102435}."
346,P4HA1,"SUBCELLULAR LOCATION: Endoplasmic reticulum lumen.,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000256|ARBA:ARBA00004319}."
347,TNXB,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}."
348,EMILIN2,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix. Note=Found mainly at the interface between amorphous elastin and microfibrils."
349,OLFM1,"SUBCELLULAR LOCATION: Secreted {ECO:0000250|UniProtKB:O88998}. Synapse {ECO:0000250|UniProtKB:O88998}. Endoplasmic reticulum {ECO:0000250|UniProtKB:O88998}. Cell projection, axon {ECO:0000250|UniProtKB:O88998}. Perikaryon {ECO:0000250|UniProtKB:O88998}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}. Synapse {ECO:0000256|ARBA:ARBA00034103}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}. Synapse {ECO:0000256|ARBA:ARBA00034103}."
350,SIL1,"SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000269|PubMed:12356756, ECO:0000269|PubMed:16282978}."
351,CRLF1,SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:9686600}.
352,SCARB2,"SUBCELLULAR LOCATION: Lysosome membrane {ECO:0000269|PubMed:17897319, ECO:0000269|PubMed:18022370, ECO:0000269|PubMed:7509809}; Multi-pass membrane protein {ECO:0000269|PubMed:17897319, ECO:0000269|PubMed:18022370, ECO:0000269|PubMed:7509809}."
353,GREM1,"SUBCELLULAR LOCATION: Secreted {ECO:0000305}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613, ECO:0000256|PIRNR:PIRNR037254}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
354,PA2G4,"SUBCELLULAR LOCATION: [Isoform 1]: Cytoplasm {ECO:0000269|PubMed:16832058, ECO:0000269|PubMed:19037095}. Nucleus, nucleolus {ECO:0000269|PubMed:16832058, ECO:0000269|PubMed:19037095}. Note=Translocates to the nucleus upon treatment with HRG. Phosphorylation at Ser-361 by PKC/PRKCD regulates its nucleolar localization. {ECO:0000269|PubMed:16832058}.; SUBCELLULAR LOCATION: [Isoform 2]: Cytoplasm {ECO:0000269|PubMed:16832058, ECO:0000269|PubMed:19037095}."
355,RPL19,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:23636399}.
356,UGGT2,"SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000255|PROSITE-ProRule:PRU10138, ECO:0000269|PubMed:12913004}. Endoplasmic reticulum-Golgi intermediate compartment {ECO:0000255|PROSITE-ProRule:PRU10138, ECO:0000269|PubMed:12913004}."
357,FKBP10,SUBCELLULAR LOCATION: Endoplasmic reticulum lumen {ECO:0000255|PROSITE-ProRule:PRU10138}.
358,COL5A3,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000255|PROSITE-ProRule:PRU00793}. Secreted {ECO:0000269|PubMed:37453717}."
359,IDH2,SUBCELLULAR LOCATION: Mitochondrion {ECO:0000250|UniProtKB:P33198}.
360,EPDR1,SUBCELLULAR LOCATION: Lysosome lumen {ECO:0000269|PubMed:30729188}. Secreted {ECO:0000269|PubMed:30729188}. Note=Lysosomal and also secreted. {ECO:0000269|PubMed:30729188}.
361,PLOD2,"SUBCELLULAR LOCATION: Rough endoplasmic reticulum membrane; Peripheral membrane protein; Lumenal side.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004367}; Peripheral membrane protein {ECO:0000256|ARBA:ARBA00004367}; Lumenal side {ECO:0000256|ARBA:ARBA00004367}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004367}; Peripheral membrane protein {ECO:0000256|ARBA:ARBA00004367}; Lumenal side {ECO:0000256|ARBA:ARBA00004367}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004367}; Peripheral membrane protein {ECO:0000256|ARBA:ARBA00004367}; Lumenal side {ECO:0000256|ARBA:ARBA00004367}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004367}; Peripheral membrane protein {ECO:0000256|ARBA:ARBA00004367}; Lumenal side {ECO:0000256|ARBA:ARBA00004367}.,SUBCELLULAR LOCATION: Endoplasmic reticulum membrane {ECO:0000256|ARBA:ARBA00004367}; Peripheral membrane protein {ECO:0000256|ARBA:ARBA00004367}; Lumenal side {ECO:0000256|ARBA:ARBA00004367}."
362,BGN,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000250}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498, ECO:0000256|RuleBase:RU364096}."
363,RERG,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000269|PubMed:11533059}.
364,PLBD2,SUBCELLULAR LOCATION: Lysosome lumen {ECO:0000269|PubMed:17105447}.
365,FBN1,"SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:24982166, ECO:0000269|PubMed:34411563}. Note=Fibrillin-1 and Asprosin chains are still linked together during the secretion from cells, but are subsequently separated by furin (PubMed:24982166). {ECO:0000269|PubMed:24982166}.; SUBCELLULAR LOCATION: [Fibrillin-1]: Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:11461921, ECO:0000269|PubMed:24982166}.; SUBCELLULAR LOCATION: [Asprosin]: Secreted {ECO:0000269|PubMed:27087445}. Note=Secreted by white adipose tissue and circulates in the plasma. {ECO:0000269|PubMed:27087445}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}.,SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000256|ARBA:ARBA00004498}."
366,PTBP1,"SUBCELLULAR LOCATION: Nucleus.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}.,SUBCELLULAR LOCATION: Nucleus {ECO:0000256|ARBA:ARBA00004123}."
367,AHSG,"SUBCELLULAR LOCATION: Secreted.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
368,ACAA1,"SUBCELLULAR LOCATION: Peroxisome {ECO:0000269|PubMed:22057399, ECO:0000269|PubMed:25538232}. Note=Transported into peroxisomes following association with PEX7. {ECO:0000269|PubMed:22057399, ECO:0000269|PubMed:25538232}."
369,PLOD1,SUBCELLULAR LOCATION: Rough endoplasmic reticulum membrane; Peripheral membrane protein; Lumenal side.
370,GUSB,SUBCELLULAR LOCATION: Lysosome.
371,BMP1,"SUBCELLULAR LOCATION: Golgi apparatus, trans-Golgi network {ECO:0000269|PubMed:12637569}. Secreted, extracellular space, extracellular matrix {ECO:0000269|PubMed:12637569}. Secreted {ECO:0000269|PubMed:33206546}. Note=Co-localizes with POSTN in the Golgi. {ECO:0000250}.; SUBCELLULAR LOCATION: [Isoform BMP1-3]: Secreted {ECO:0000269|PubMed:21453682}."
372,LOX,"SUBCELLULAR LOCATION: Secreted, extracellular space {ECO:0000256|ARBA:ARBA00004239, ECO:0000256|RuleBase:RU367046}.,SUBCELLULAR LOCATION: Secreted {ECO:0000269|PubMed:31152061}. Secreted, extracellular space.,SUBCELLULAR LOCATION: Secreted, extracellular space {ECO:0000256|RuleBase:RU367046}.,SUBCELLULAR LOCATION: Secreted, extracellular space {ECO:0000256|ARBA:ARBA00004239, ECO:0000256|RuleBase:RU367046}.,SUBCELLULAR LOCATION: Secreted, extracellular space {ECO:0000256|ARBA:ARBA00004239, ECO:0000256|RuleBase:RU367046}."
373,MFAP5,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix."
374,LAMB3,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix, basement membrane."
375,C3,"SUBCELLULAR LOCATION: Secreted.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}.,SUBCELLULAR LOCATION: Secreted {ECO:0000256|ARBA:ARBA00004613}."
376,ST6GALNAC3,SUBCELLULAR LOCATION: Golgi apparatus membrane {ECO:0000305}; Single-pass type II membrane protein {ECO:0000305}.
377,COL1A2,"SUBCELLULAR LOCATION: Secreted, extracellular space, extracellular matrix {ECO:0000255|PROSITE-ProRule:PRU00793}."
378,F13A1,"SUBCELLULAR LOCATION: Cytoplasm. Secreted {ECO:0000269|PubMed:4405643}. Note=Secreted into the blood plasma. Cytoplasmic in most tissues, but also secreted in the blood plasma."
'''

In [ ]:
gene_pattern = re.compile(r'\d+,([A-Za-z0-9]+),')
non_membrane_genes = gene_pattern.findall(string)

In [ ]:
# Find all the surfaces genes appearing in original dataset
selected = set()
for gene, nTPM in cleaned_data[0][2]:
  selected.add(gene)

# Remove non-membrane genes
for gene in selected.copy():
    if gene in non_membrane_genes:
        selected.remove(gene)

# Form the cleaned gene expression table
tissue_cells = []
nTPM_matrix = []
m = len(cleaned_data)
n = len(selected)
gene_list = list(selected)
for i in range(m):
  row = cleaned_data[i]
  tissue_cells.append((row[0], row[1]))
  zeros = [0 for _ in range(n)]
  for gene, nTPM in row[2]:
    if gene in gene_list:
      j = gene_list.index(gene)
      zeros[j] = nTPM
  nTPM_matrix.append(zeros)

# write the gene expression table into csv file
file_path = 'gene_expression_matrix.csv'
with open(file_path, 'w') as file:
    for row in nTPM_matrix:
      line = '\t'.join(map(str, row))
      file.write(line + '\n')

# write the cell types appearing in original dataset into tsv file as format (tissue, cell)
file_path = 'tissue_cell_pairs.tsv'
with open(file_path, 'w') as file:
    for name in tissue_cells:
        line = '\t'.join(map(str, name))
        file.write(line + '\n')

# write all the surfaces genes appearing in original dataset into csv file
file_path = 'gene_list.csv'
with open(file_path, 'w') as file:
    for gene in gene_list:
      file.write(gene + '\n')

# Form Labels

Create Positive Labels

In [ ]:
Cell_marker = pd.read_excel('Cell_marker_Human.xlsx')
PanglaoDB = pd.read_csv('PanglaoDB_markers_27_Mar_2020.tsv', delimiter='\t')

Count markers in CellMarker and Panglao

In [ ]:
# Cellmarker
matched_cells1 = {}
name_no_match1 = set()
file_path_1 = 'CellMarker_name_match.csv'
with open(file_path_1 , 'r') as file:
    reader = csv.reader(file, delimiter='\t')
    next(reader, None)
    for row in reader:
      formatted = ast.literal_eval(row[0].replace('‘', "'").replace('’', "'").replace('\\', ''))
      if len(formatted) > 1:
        if formatted[1] == None:
          name_no_match1.add(formatted[0])
        else:
          matched_cells1[formatted[0]] = ast.literal_eval(formatted[1])
      else:
        name_no_match1.add(formatted[0])

marker_count1 =0
for key in matched_cells1:
  tissue, cell = matched_cells1[key]
  genes = Cell_marker[(Cell_marker['species'] == 'Human') & (Cell_marker['tissue_type'] == tissue) & (Cell_marker['cell_name'] == cell)]['Symbol'].tolist()
  genes = list(set(genes))
  marker_count1 += len(genes)
  matched_cells1[key] = genes

print("number of cells in CellMarker matched with dataset:", len(matched_cells1))
print("number of cells in CellMarker not matched with dataset:", len(name_no_match1))
print("number of markers:", marker_count1)

# Panglao
matched_cells2 = {}
name_no_match2 = set()
file_path_2 = 'PanglaoDB_name_match.csv'
with open(file_path_2 , 'r') as file:
    reader = csv.reader(file, delimiter='\t')
    next(reader, None)
    for row in reader:
      formatted = ast.literal_eval(row[0].replace('‘', "'").replace('’', "'").replace('\\', ''))
      if formatted[1] == None:
        name_no_match2.add(formatted[0])
      else:
        matched_cells2[formatted[0]] = ast.literal_eval(formatted[1])

marker_count2 =0
for key in matched_cells2:
  tissue, cell = matched_cells2[key]
  genes = PanglaoDB[(PanglaoDB['species'] != 'Mm') & (PanglaoDB['organ'] == tissue) & (PanglaoDB['cell type'] == cell)]['official gene symbol'].tolist()
  genes = list(set(genes))
  marker_count2 += len(genes)
  matched_cells2[key] = genes

print("number of cells in Panglap matched with dataset:", len(matched_cells2))
print("number of cells in Panglao not matched with dataset:", len(name_no_match2))
print("number of markers:", marker_count2)

# Combine collected markers from CellMarker and Panglao
total_count = 0
markers = {}
for key in matched_cells2:
  if key in matched_cells1:
    genes = matched_cells1[key] + matched_cells2[key]
    genes = list(set(genes))
  else:
    genes = matched_cells2[key]
  markers[key] = genes
  total_count += len(genes)
for key in matched_cells1:
  if key not in markers:
    genes = matched_cells1[key]
    markers[key] = genes
    total_count += len(genes)

print("number of cells in combined set:", len(markers))
print("number of total markers:", total_count)

number of cells in CellMarker matched with dataset: 48
number of cells in CellMarker not matched with dataset: 10
number of markers: 1124
number of cells in Panglap matched with dataset: 44
number of cells in Panglao not matched with dataset: 185
number of markers: 2754
number of cells in combined set: 79
number of total markers: 3718


Form the positive labels

In [ ]:
positives = {}
for key in markers:
  i = tissue_cells.index(key)
  positives[i] = []
  genes = markers[key]
  for gene in genes:
    if gene in gene_list:
      j = gene_list.index(gene)
      positives[i].append(j)
  if positives[i] == []:
    del positives[i]

count1 = 0
for i in positives:
  count1 += len(positives[i])

print("number of cells in positive labels:", len(positives))
print("number of positive labels:", count1)

Form the negative labels

In [ ]:
# Form negative lables
negatives = {}
for i in positives:
  name = tissue_cells[i]
  tissue = name[0]
  cell = name[1]
  for j in range(len(tissue_cells)):
    t = tissue_cells[j][0]
    c = tissue_cells[j][1]
    if t == tissue and c != cell:
      if j in negatives:
        negatives[j] = list(set(negatives[j] + positives[i]))
      else:
        negatives[j] = positives[i]

count2 = 0
for i in negatives:
  count2 += len(negatives[i])

print("number of cells in negative labels:", len(negatives))
print("number of negative labels:", count2)

number of cells in negative labels: 179
number of negative labels: 3992


In [ ]:
# write the postive labels into file
file_path = 'positive_labels.csv'
with open(file_path , mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["cell_name_id", "gene_id"])
    for key, values in positives.items():
      if values != []:
        writer.writerow([key] + values)

# write the negative labels into file
file_path = 'negative_labels.csv'
with open(file_path , mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["cell_name_id", "gene_id"])
    for key, values in negatives.items():
      if values != []:
        writer.writerow([key] + values)